# README

*   Pada file ops/make.sh ganti path yang sesuai
*   Bila menggunakan jupyter/colab setelah melakukan instalasi package harap restart kernel
*   Untuk pergantian path tedapat pada tab Parameters
*   Perubahan dataset dan model terdapat pada setiap tab experiments (K-Fold dan Cross dataset)
*   NOTE: bila menggunakan deterministic, variable lr_latent diganti menjadi None dan bila menggunakan probabilistic, vaiable lr_latent diganti menjadi 0.001

# Install

In [ ]:
!pip install -r drive/MyDrive/Thesis/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 7.7 MB/s 
     |████████████████████████████████| 145 kB 56.8 MB/s 
     |████████████████████████████████| 178 kB 70.0 MB/s 
     |████████████████████████████████| 829 kB 87.8 MB/s 
     |████████████████████████████████| 147 kB 77.9 MB/s 
     |████████████████████████████████| 4.2 MB 61.4 MB/s 
     |████████████████████████████████| 303 kB 73.4 MB/s 
     |████████████████████████████████| 97 kB 7.3 MB/s 
     |████████████████████████████████| 313 kB 74.9 MB/s 
     |████████████████████████████████| 497 kB 53.6 MB/s 
     |████████████████████████████████| 14.8 MB 64.3 MB/s 
     |████████████████████████████████| 146 kB 56.3 MB/s 
     |████████████████████████████████| 49.9 MB 77.9 MB/s 
     |████████████████████████████████| 3.0 MB 62.2 MB/s 
     |████████████████████████████████| 636 kB 53.0 MB/s 
     |██████████████████████████

In [ ]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 82.7 MB/s eta 0:00:04tcmalloc: large alloc 1147494400 bytes == 0x39346000 @  0x7f6ec2b24615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:27tcmalloc: large alloc 1434370048 bytes == 0x7d99c000 @  0x7f6ec2b24615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████

In [ ]:
!bash drive/MyDrive/Thesis/ops/make.sh
!pip install MultiScaleDeformableAttention

running build
running build_ext
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:352: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
building 'MultiScaleDeformableAttention' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/drive
creating build/temp.linux-x86_64-3.7/content/drive/MyDrive
creating build/temp.linux-x86_64-3.7/content/drive/MyDrive/Thesis
creating build/temp.linux-x86_64-3.7/content/drive/MyDrive/Thesis/ops
creating build/temp.linux-x86_64-3.7/content/drive/MyDrive/Thesis/ops/src
creating build/temp.linux-x86_64-3.7/content/drive/MyDrive/Thesis/ops/src/cpu
creating build/temp.linux-x86_64-3.7/content/drive/MyDrive/Thesis/ops/src/cuda
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DND

# Import

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import re
import shutil
import tensorflow as tf
import pandas as pd
import numpy as np
import nibabel as nib
from glob import glob
import albumentations as A
from keras import backend as K
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
import copy
import math
import torch
import warnings
import torch.nn as nn
import torch.nn.functional as F

from mmcv.cnn import normal_init
from typing import Optional
from torch import nn, Tensor
from mmcv.cnn import ConvModule
from torch.nn.init import normal_
from torch.autograd import Variable
from torch.nn.init import xavier_uniform_, constant_
from mmcv.cnn import constant_init, kaiming_init, build_conv_layer, build_norm_layer
from mmcv.runner import load_checkpoint
from torch.nn.modules.batchnorm import _BatchNorm
import logging

from mmcv.utils import get_logger
import MultiScaleDeformableAttention as MSDA
from torch.autograd import Function
from torch.autograd.function import once_differentiable

# 1. Helpers

In [ ]:
def read_data(file_path, start=0, end=0, step=1):
    img = nib.load(file_path)
    data = img.get_fdata()
    end = data.shape[-1] if end == 0 else end
    reverse_channel = [data[:,:,i] for i in range(start,end,step)]
    return np.array(reverse_channel, dtype='float32')

def adjust_data(img, cf=1, mask=False):
    if mask is False:
        # Normalized data 
        img = (img - np.mean(img))/(np.std(img) + K.epsilon())

    img = np.expand_dims(img, -1)
    # Central crop if needed
    if cf<1: img = tf.image.central_crop(img, cf)
    return img

def check_data(img, seg):
    # Select data that have a value (image that contain only background)
    idx = [k for k,i in enumerate(img) if len(np.unique(i)) > 1] 
    return img[idx], seg[idx]

def data_augmentation(images, masks):
    transform = A.Compose([A.HorizontalFlip(), A.Rotate(p=0.8)])
    for k,(i,j) in enumerate(zip(images,masks)):
        transformed = transform(image=i, mask=j)
        images[k] = transformed['image']
        masks[k] = transformed['mask']

def find_best(path_model, type_='min', model_type='det'):
    # Find best weights in filename
    if model_type == 'det':
        path = glob(path_model+'/*.hdf5') 
    else:
        path = glob(path_model+'/*') 
        
    split = [i.split('-')[-1] for i in path]
    split.sort(key=natural_keys)

    if type_ == 'min':
        file_ = split[0]
    else:
        file_ = split[-1]
    return [i for i in path if file_ in i] [0]

def show_img(img, y_true, y_pred, idx=0):
    # Visualize image, ground truth, and prediction by slice
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1); plt.title('FLAIR image')
    plt.imshow(img[idx,:,:,0], cmap='gray'); plt.axis('off')
    plt.subplot(1,3,2); plt.title('Ground Truth')
    plt.imshow(y_true[idx,:,:,0], cmap='gray'); plt.axis('off')
    plt.subplot(1,3,3); plt.title('Prediction')
    plt.imshow(y_pred[idx,:,:,0], cmap='gray'); plt.axis('off')
    plt.show()

def save_wmh(y_pred, file_in, file_out, name='ADNI'):
    # Save WMHs predictions 
    y_true = nib.load(file_in)
    wmh = np.zeros(y_true.shape)

    if name == 'Singapore':
        if wmh.shape[1] != y_pred.shape[2]:
            wmh_temp = tf.image.rot90(y_pred, k=3).numpy()
            for i in range(wmh.shape[-1]):
                wmh[:,12:220,i] = wmh_temp[i,:,:,0]
        else:
            for i in range(wmh.shape[-1]):
                wmh[12:220,:,i] = y_pred[i,:,:,0]

    elif name == 'GE3T':
        for i in range(wmh.shape[-1]):
            wmh[:128,:,i] = y_pred[i,:,:,0]
    else:
        for i in range(wmh.shape[-1]):
            wmh[:,:,i] = y_pred[i,:,:,0]

    wmh = nib.Nifti1Image(wmh, y_true.affine, y_true.header)
    nib.save(wmh, file_out)

def save_wmh_challenge(y_pred, file_in, file_out, name):
    # Save WMHs prediction only for Challenge dataset as full
    y_true = nib.load(file_in)
    wmh = np.zeros(y_true.shape)

    if name == 'Singapore': start=4; end=236
    elif name == 'GE3T': start=54; end=186
    elif name == 'Utrecht': start=8; end=248
    else: start=0; end=240

    if name == 'Singapore':
        if wmh.shape[1] != y_pred.shape[2]:
            wmh_temp = tf.image.rot90(y_pred, k=3).numpy()
            for i in range(wmh.shape[-1]):
                wmh[:,:,i] = wmh_temp[i,:,start:end,0]
        else:
            for i in range(wmh.shape[-1]):
                wmh[:,:,i] = y_pred[i,start:end,:,0]

    elif name == 'Utrecht':
        for i in range(wmh.shape[-1]):
            wmh[:,:,i] = y_pred[i,:,start:end,0]
    else:
        for i in range(wmh.shape[-1]):
            wmh[:,:,i] = y_pred[i,start:end,:,0]

    wmh = nib.Nifti1Image(wmh, y_true.affine, y_true.header)
    nib.save(wmh, file_out)

def raw2nii(type_, train=True, bias_cor=True, path_dataset=None):
    # Convert .raw into .nii.gz
    train = 'training' if train else 'testing'
    path_files = path_dataset+'NITRC/'+train+'/{}/{}'
    path_raw = glob(path_files.format('*', '*'+type_+'.nhdr'))
    for i in path_raw:
        file_name = i.split('/')[-1].split('.')[0] + '.nii.gz'
        if bias_cor:
            bias_field_correction(i, path_files.format('full',file_name))
        else:
            img = sitk.ReadImage(i)
            sitk.WriteImage(img, path_files.format('full',file_name))

def bias_field_correction(path, out):
    # BFC using ANTs library
    img = nib.load(path)
    bfc = ants.image_read(path)
    bfc = ants.n3_bias_field_correction(bfc)
    bfc = nib.Nifti1Image(bfc.numpy(), img.affine, img.header)
    nib.save(bfc, out)

# https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
def atoi(text):
    return int(text) if text.isdigit() else text

# https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

# https://github.com/MrGiovanni/UNetPlusPlus/blob/master/keras/helper_functions.py#L37-L42
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# https://github.com/umbertogriffo/focal-loss-keras/blob/master/src/loss_function/losses.py#L11-L53
def binary_focal_loss(gamma=2., alpha=.25):
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        y_true = tf.cast(y_true, tf.float32)
        # Define epsilon so that the back-propagation will not result in NaN for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        # y_pred = y_pred + epsilon
        # Clip the prediciton value
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        # Calculate alpha_t
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        # Calculate cross entropy
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1 - p_t), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.mean(K.sum(loss, axis=1))
        return loss
    return binary_focal_loss_fixed

# modified code from https://stackoverflow.com/questions/53303724/how-to-average-only-non-zero-entries-in-tensor
# and https://stackoverflow.com/questions/42606207/keras-custom-decision-threshold-for-precision-and-recall
def volume_loss(threshold=0.25):
    def calculate(y_true, y_pred):
        nonzero_true = K.any(K.not_equal(y_true, 0), axis=-1)
        y_true_volume = K.sum(K.cast(nonzero_true, 'float32'), axis=-1, keepdims=True)
        y_true_volume = K.sum(y_true_volume/1000)

        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold), K.floatx())
        nonzero_pred = K.any(K.not_equal(y_pred, 0), axis=-1)
        y_pred_volume = K.sum(K.cast(nonzero_pred, 'float32'), axis=-1, keepdims=True)
        y_pred_volume = K.sum(y_pred_volume/1000)
        
        return K.abs(y_true_volume-y_pred_volume)
    return calculate

def total_loss(type_loss='FL', gamma=2., alpha=.25, threshold=0.25):
    def calculate(y_true, y_pred):
        if type_loss == 'FTL':
            loss = focal_tversky(gamma=gamma, alpha=alpha) (y_true, y_pred)
        else:
            loss = binary_focal_loss(gamma=gamma, alpha=alpha) (y_true, y_pred)
        vl = volume_loss(threshold=threshold) (y_true, y_pred)
        
        return loss+vl
    return calculate

# https://github.com/baumgach/PHiSeg-code/blob/c43f3b32e1f434aecba936ff994b6f743ba7a5f8/utils.py#L326-L370
def ambiguity_map(y_gen, seg=None):
    def pixel_wise_xent(m_samp, m_gt, eps=1e-8):
        log_samples = np.log(m_samp + eps)
        return -1.0*np.sum(m_gt*log_samples, axis=-1)

    y_pred = np.average(y_gen, axis=0)
    E_arr = np.zeros(y_gen.shape)
    for i in range(y_gen.shape[0]):
        for j in range(y_gen.shape[1]):
            if seg is None:
                E_arr[i,j,...] = np.expand_dims(pixel_wise_xent(y_gen[i,j,...], y_pred[j,...]), axis=-1)
            else:
                E_arr[i,j,...] = np.expand_dims(pixel_wise_xent(y_gen[i,j,...], seg[j,...]), axis=-1)

    return np.average(E_arr, axis=0)

# 2. Datasets

In [ ]:
def get_dataset(name, end=0):
    if name == 'ADNI' and end >= 1:
        start = 5; end = 30; step=1
    elif name == 'NITRC'and end >= 1:
        start = 140; end = 350; step=5
    elif name == 'Singapore' and end >= 1:
        start = 0; end = 45; step=1
    elif name == 'GE3T' and end >= 1:
        start = 25; end = 70; step=1
    elif name == 'Utrecht' and end >= 1:
        start = 0; end = 45; step=1
    else:
        start = 0; end = 0; step=1

    if name == 'Challenge':
        fold_img, fold_seg = read_challenge(start, end, step=1, cf=1)
    else:
        fold_img, fold_seg = read_dataset(name, start, end, step=step, cf=1)
    
    return fold_img, fold_seg

def read_dataset(name, start, end, step=1, cf=1):
    if name == 'ADNI':
        flair_path = glob(path_dataset+'ADNI/*/*/*brain.nii.gz')
        lesion_path = glob(path_dataset+'ADNI/*/*/*wmh.nii.gz')
    elif name == 'NITRC':
        flair_path = glob(path_dataset+'NITRC/training/*/*brain.nii.gz')
        lesion_path = glob(path_dataset+'NITRC/training/*/*lesion.nii.gz')
    else:
        flair_path = glob(path_dataset+'Challenge/{}/*/*brain.nii.gz'.format(name))
        lesion_path = glob(path_dataset+'Challenge/{}/*/*wmh.nii.gz'.format(name))

    flair_path.sort(key=natural_keys) 
    lesion_path.sort(key=natural_keys)
    
    if name == 'ADNI':
        flair_path = [[i for i in flair_path if i.split('/')[-3] == 'fold'+str(j)] for j in range(1,5)]
        lesion_path = [[i for i in lesion_path if i.split('/')[-3] == 'fold'+str(j)] for j in range(1,5)]
    else:
        flair_path = [flair_path]
        lesion_path = [lesion_path]

    fold_img, fold_seg = [], []
    for fp,lp in zip(flair_path,lesion_path):
        if name == 'Singapore':
            # Adjust first 2 data at Singapore
            img_temp = [np.expand_dims(read_data(i, start=start, end=end, step=step), -1) for i in fp[:2]]
            seg_temp = [np.expand_dims(read_data(i, start=start, end=end, step=step), -1) for i in lp[:2]]
            img_temp = np.array([[np.squeeze(tf.image.rot90(j).numpy(), -1) for j in i] for i in img_temp])
            seg_temp = np.array([[np.squeeze(tf.image.rot90(j).numpy(), -1) for j in i] for i in seg_temp])
            del fp[:2]; del lp[:2]

        # Read Data
        img = np.array([read_data(i, start=start, end=end, step=step) for i in fp])
        seg = np.array([read_data(i, start=start, end=end, step=step) for i in lp])

        if name == 'Singapore':
            # Concat with first 2 data and cropping
            img = np.concatenate([img_temp, img])
            seg = np.concatenate([seg_temp, seg])
            img = np.array([i[:,12:220,:] for i in img])
            seg = np.array([i[:,12:220,:] for i in seg])
        elif name == 'GE3T':
            # Cropping
            img = np.array([i[:,:128,:] for i in img])
            seg = np.array([i[:,:128,:] for i in seg])

        # Normalized data
        img = np.array([[adjust_data(j, cf=cf) for j in i] for i in img])
        seg = np.array([[adjust_data(j, cf=cf, mask=True) for j in i] for i in seg])

        if name not in ['ADNI', 'NITRC']:
            seg[(seg == 2).all(axis=-1)] = 0

        fold_img.append(img); fold_seg.append(seg)
    return np.array(fold_img), np.array(fold_seg)

def read_challenge(start, end, step=1, cf=1):
    # Read Challenge dataset as full dataset
    fold_img, fold_seg = [], []
    for name in ['Singapore', 'GE3T', 'Utrecht']:
        flair_path = glob(path_dataset+'Challenge/{}/*/*brain.nii.gz'.format(name))
        lesion_path = glob(path_dataset+'Challenge/{}/*/*wmh.nii.gz'.format(name))
        flair_path.sort(key=natural_keys) 
        lesion_path.sort(key=natural_keys)
        flair_path = [flair_path]
        lesion_path = [lesion_path]

        for fp,lp in zip(flair_path, lesion_path):
            if name == 'Singapore':
                # Adjust first 2 data at Singapore
                img_temp = [np.expand_dims(read_data(i, start=start, end=end, step=step), -1) for i in fp[:2]]
                seg_temp = [np.expand_dims(read_data(i, start=start, end=end, step=step), -1) for i in lp[:2]]
                img_temp = np.array([[np.squeeze(tf.image.rot90(j).numpy(), -1) for j in i] for i in img_temp])
                seg_temp = np.array([[np.squeeze(tf.image.rot90(j).numpy(), -1) for j in i] for i in seg_temp])
                del fp[:2]; del lp[:2]

            # Read Data
            img = np.array([read_data(i, start=start, end=end, step=step) for i in fp])
            seg = np.array([read_data(i, start=start, end=end, step=step) for i in lp])
            
            if name == 'Singapore':
                img = np.concatenate([img_temp, img])
                seg = np.concatenate([seg_temp, seg])
            fold_img.append(img); fold_seg.append(seg)

    # Get the max sizes from all insitutions
    sizes = [i.shape[-2:] for i in fold_img]
    max_sizes = np.max(list(zip(*sizes)), -1).tolist()
    diff_sizes = [(int((max_sizes[0]-i)/2), (int((max_sizes[1]-j)/2))) for i,j in sizes]
    fold_img_padd, fold_seg_padd = [], []

    for k,(i,j) in enumerate(zip(fold_img, fold_seg)):
        # Zero padding
        img = np.zeros(i.shape[:2] + tuple(max_sizes))
        seg = np.zeros(i.shape[:2] + tuple(max_sizes))
        x = sizes[k][0]; x_pad = diff_sizes[k][0]
        y = sizes[k][1]; y_pad = diff_sizes[k][1]

        img[:,:,x_pad:x+x_pad,y_pad:y+y_pad] = i
        seg[:,:,x_pad:x+x_pad,y_pad:y+y_pad] = j

        # Normalized data
        img = np.array([[adjust_data(j, cf=cf) for j in i] for i in img])
        seg = np.array([[adjust_data(j, cf=cf, mask=True) for j in i] for i in seg])
        seg[(seg == 2).all(axis=-1)] = 0
        fold_img_padd.append(img); fold_seg_padd.append(seg)

    return fold_img_padd, fold_seg_padd

# 3. Parameters

In [ ]:
# Paths config
## For saving WMHs prediction
result_path = '../content/drive/MyDrive/Thesis/Results/{}/{}/{}'
## Pre-trained path
path = '../content/drive/MyDrive/Publikasi/pre-trained/'
## Dataset path
path_dataset = '../content/drive/MyDrive/Datasets/'
## Evaluation path
eval_path = '../content/drive/MyDrive/Thesis/Evaluation_Excel/'

dataset_type = ['ADNI', 'Singapore', 'GE3T', 'Utrecht', 'Challenge']
# reduce_lr = ReduceLROnPlateau(monitor='val_total_loss', factor=0.5, patience=2)
generate_pred = 30
threshold = 0.25
batch_size = 4
epoch = 100
lr = 0.001

lr_latent = None#0.001
# Deterministic gamma = 1.0, Probabilistic gamma = 0.25
gamma = 1.0
alpha = 0.5
beta = 1
if lr_latent is None:
    uniq = str(lr)+'_'+str(gamma)+'_'+str(alpha)
else:
    uniq = str(lr)+'_'+str(lr_latent)+'_'+str(gamma)+'_'+str(alpha)

def pick_model(model_name, dataset_name=None):
    n_blocks = 4
    n_classes = 1 
    input_shape = (None, None, 1)
    decoder_filters = (256, 128, 64, 32)

    # Deterministic
    if model_name == 'U_Net':
        md = Unet(use_backbone=False, input_shape=input_shape, attention=False,
                n_upsample_blocks=n_blocks, decoder_filters=decoder_filters)
    
    elif model_name == 'MCTrans':
        md = EncoderDecoder(n_points=6)

    # Probabilistic
    elif model_name == 'Prob_U_Net':
        md = Prob_Unet(num_classes=n_classes, activation='sigmoid', latent_dim=6, concat_lvl=[0])
    
    elif model_name == 'Prob_MCTrans':
        md = EncoderDecoder(prob=True, n_points=4, latent_dim=6)
        
    else:
        md = None
    return md

# MCTrans

## Utils

In [ ]:
def resize(input,
           size=None,
           scale_factor=None,
           mode='nearest',
           align_corners=None,
           warning=True):
    if warning:
        if size is not None and align_corners:
            input_h, input_w = tuple(int(x) for x in input.shape[2:])
            output_h, output_w = tuple(int(x) for x in size)
            if output_h > input_h or output_w > output_h:
                if ((output_h > 1 and output_w > 1 and input_h > 1
                     and input_w > 1) and (output_h - 1) % (input_h - 1)
                        and (output_w - 1) % (input_w - 1)):
                    warnings.warn(
                        f'When align_corners={align_corners}, '
                        'the output would more aligned if '
                        f'input size {(input_h, input_w)} is `x+1` and '
                        f'out size {(output_h, output_w)} is `nx+1`')
    if isinstance(size, torch.Size):
        size = tuple(int(x) for x in size)
    if isinstance(size, (numpy.ndarray, numpy.generic) ):
        size = tuple(int(x) for x in size)
    return F.interpolate(input, size, scale_factor, mode, align_corners)

def get_root_logger(log_file=None, log_level=logging.INFO):
    """Get the root logger.
    The logger will be initialized if it has not been initialized. By default a
    StreamHandler will be added. If `log_file` is specified, a FileHandler will
    also be added. The name of the root logger is the top-level package name,
    e.g., "mmseg".
    Args:
        log_file (str | None): The log filename. If specified, a FileHandler
            will be added to the root logger.
        log_level (int): The root logger level. Note that only the process of
            rank 0 is affected, while other processes will set the level to
            "Error" and be silent most of the time.
    Returns:
        logging.Logger: The root logger.
    """

    logger = get_logger(name='MCTrans', log_file=log_file, log_level=log_level)

    return logger

## Full

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, num_classes=1, d_model=128, num_filters=[64,64,128,128,128], n_points=4, prob=False, latent_dim=6):
        super(EncoderDecoder, self).__init__()
        self.prob = prob
        self.encoder = ResNet(in_channels=1, depth=18, out_indices=(0, 1, 2, 3, 4))
        self.decoder = UNetDecoder(in_channels=num_filters)

        self.center = MCTrans(d_model=d_model, nhead=8, d_ffn=512, dropout=0.1, 
                              act="relu", n_levels=3, n_points=n_points, n_sa_layers=6)
        
        self.head = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=1)
        self.aux_head = MCTransAuxHead(d_model=d_model, d_ffn=512, act="relu", 
                                       num_classes=num_classes, in_channles=num_filters)

        if self.prob:
            self.initializers = {'w':'he_normal', 'b':'normal'}
            self.prior = AxisAlignedConvGaussian(input_channels=1, num_filters=num_filters[:-1], 
                                                no_convs_per_block=2, latent_dim=latent_dim,
                                                 initializers=self.initializers)
            self.posterior = AxisAlignedConvGaussian(input_channels=1, num_filters=num_filters[:-1],
                                                    no_convs_per_block=2, latent_dim=latent_dim,
                                                    initializers=self.initializers, posterior=True)
            self.fcomb = Fcomb(num_filters[:-1], latent_dim, num_classes, num_classes, 2, self.initializers, use_tile=True)

    def forward(self, img, seg):
        x = self.extract_feat(img)
        x_ = self.decoder(x)
        x_ = self.head(x_)
        logits = self.aux_head(x)
        
        if self.prob:
            self.posterior_latent_space = self.posterior(img, seg)
            self.prior_latent_space = self.prior(img)
            x_ = self.fcomb(x_, self.posterior_latent_space.rsample())

        return x_, logits

    def sampling(self, img, seg):
        x = self.extract_feat(img)
        x_ = self.decoder(x)
        x_ = self.head(x_)
        logits = self.aux_head(x)
        
        if self.prob:
            self.posterior_latent_space = self.posterior(img, seg)
            self.prior_latent_space = self.prior(img)
            x_ = self.fcomb(x_, self.prior_latent_space.sample())

        return x_, logits

    def extract_feat(self, img):
        x = self.encoder(img)
        x = self.center(x)
        return x

    def kl_divergence(self, analytic=False, calculate_posterior=True, z_posterior=None):
        """
        Calculate the KL divergence between the posterior and prior KL(Q||P)
        analytic: calculate KL analytically or via sampling from the posterior
        calculate_posterior: if we use samapling to approximate KL we can sample here or supply a sample
        """
        if analytic:
            # Need to add this to torch source code, see: https://github.com/pytorch/pytorch/issues/13545
            kl_div = kl._kl_independent_independent(self.posterior_latent_space, self.prior_latent_space)
        else:
            if calculate_posterior:
                z_posterior = self.posterior_latent_space.rsample()
            log_posterior_prob = self.posterior_latent_space.log_prob(z_posterior)
            log_prior_prob = self.prior_latent_space.log_prob(z_posterior)
            kl_div = log_posterior_prob - log_prior_prob
        return kl_div

## Probabilistic

In [ ]:
# https://github.com/stefanknegt/Probabilistic-Unet-Pytorch/blob/master/probabilistic_unet.py

from torch.distributions import Normal, Independent

class Encoder(nn.Module):
    """
    A convolutional neural network, consisting of len(num_filters) times a block of no_convs_per_block convolutional layers,
    after each block a pooling operation is performed. And after each convolutional layer a non-linear (ReLU) activation function is applied.
    """
    def __init__(self, input_channels, num_filters, no_convs_per_block, initializers, padding=True, posterior=False):
        super(Encoder, self).__init__()
        self.contracting_path = nn.ModuleList()
        self.input_channels = input_channels
        self.num_filters = num_filters

        if posterior:
            #To accomodate for the mask that is concatenated at the channel axis, we increase the input_channels.
            self.input_channels += 1

        layers = []
        for i in range(len(self.num_filters)):
            """
            Determine input_dim and output_dim of conv layers in this block. The first layer is input x output,
            All the subsequent layers are output x output.
            """
            input_dim = self.input_channels if i == 0 else output_dim
            output_dim = num_filters[i]
            
            if i != 0:
                layers.append(nn.AvgPool2d(kernel_size=2, stride=2, padding=0, ceil_mode=True))
            
            layers.append(nn.Conv2d(input_dim, output_dim, kernel_size=3, padding=int(padding)))
            layers.append(nn.ReLU(inplace=True))

            for _ in range(no_convs_per_block-1):
                layers.append(nn.Conv2d(output_dim, output_dim, kernel_size=3, padding=int(padding)))
                layers.append(nn.ReLU(inplace=True))

        self.layers = nn.Sequential(*layers)

        self.layers.apply(init_weights)

    def forward(self, input):
        output = self.layers(input)
        return output

class AxisAlignedConvGaussian(nn.Module):
    """
    A convolutional net that parametrizes a Gaussian distribution with axis aligned covariance matrix.
    """
    def __init__(self, input_channels, num_filters, no_convs_per_block, latent_dim, initializers, posterior=False):
        super(AxisAlignedConvGaussian, self).__init__()
        self.input_channels = input_channels
        self.channel_axis = 1
        self.num_filters = num_filters
        self.no_convs_per_block = no_convs_per_block
        self.latent_dim = latent_dim
        self.posterior = posterior
        if self.posterior:
            self.name = 'Posterior'
        else:
            self.name = 'Prior'
        self.encoder = Encoder(self.input_channels, self.num_filters, self.no_convs_per_block, initializers, posterior=self.posterior)
        self.conv_layer = nn.Conv2d(num_filters[-1], 2 * self.latent_dim, (1,1), stride=1)
        self.show_img = 0
        self.show_seg = 0
        self.show_concat = 0
        self.show_enc = 0
        self.sum_input = 0

        nn.init.kaiming_normal_(self.conv_layer.weight, mode='fan_in', nonlinearity='relu')
        nn.init.normal_(self.conv_layer.bias)

    def forward(self, input, segm=None):

        #If segmentation is not none, concatenate the mask to the channel axis of the input
        if segm is not None:
            self.show_img = input
            self.show_seg = segm
            input = torch.cat((input, segm), dim=1)
            self.show_concat = input
            self.sum_input = torch.sum(input)

        encoding = self.encoder(input)
        self.show_enc = encoding

        #We only want the mean of the resulting hxw image
        encoding = torch.mean(encoding, dim=2, keepdim=True)
        encoding = torch.mean(encoding, dim=3, keepdim=True)

        #Convert encoding to 2 x latent dim and split up for mu and log_sigma
        mu_log_sigma = self.conv_layer(encoding)

        #We squeeze the second dimension twice, since otherwise it won't work when batch size is equal to 1
        mu_log_sigma = torch.squeeze(mu_log_sigma, dim=2)
        mu_log_sigma = torch.squeeze(mu_log_sigma, dim=2)

        mu = mu_log_sigma[:,:self.latent_dim]
        log_sigma = mu_log_sigma[:,self.latent_dim:]

        #This is a multivariate normal with diagonal covariance matrix sigma
        #https://github.com/pytorch/pytorch/pull/11178
        dist = Independent(Normal(loc=mu, scale=torch.exp(log_sigma)),1)
        return dist

class Fcomb(nn.Module):
    """
    A function composed of no_convs_fcomb times a 1x1 convolution that combines the sample taken from the latent space,
    and output of the UNet (the feature map) by concatenating them along their channel axis.
    """
    def __init__(self, num_filters, latent_dim, num_output_channels, num_classes, no_convs_fcomb, initializers, use_tile=True):
        super(Fcomb, self).__init__()
        self.num_channels = num_output_channels #output channels
        self.num_classes = num_classes
        self.channel_axis = 1
        self.spatial_axes = [2,3]
        self.num_filters = num_filters
        self.latent_dim = latent_dim
        self.use_tile = use_tile
        self.no_convs_fcomb = no_convs_fcomb 
        self.name = 'Fcomb'

        if self.use_tile:
            layers = []

            #Decoder of N x a 1x1 convolution followed by a ReLU activation function except for the last layer
            layers.append(nn.Conv2d(self.num_channels+self.latent_dim, self.num_filters[0], kernel_size=1))
            layers.append(nn.ReLU(inplace=True))

            for _ in range(no_convs_fcomb-2):
                layers.append(nn.Conv2d(self.num_filters[0], self.num_filters[0], kernel_size=1))
                layers.append(nn.ReLU(inplace=True))

            self.layers = nn.Sequential(*layers)

            self.last_layer = nn.Conv2d(self.num_filters[0], self.num_classes, kernel_size=1)

            if initializers['w'] == 'orthogonal':
                self.layers.apply(init_weights_orthogonal_normal)
                self.last_layer.apply(init_weights_orthogonal_normal)
            else:
                self.layers.apply(init_weights)
                self.last_layer.apply(init_weights)

    def tile(self, a, dim, n_tile):
        """
        This function is taken form PyTorch forum and mimics the behavior of tf.tile.
        Source: https://discuss.pytorch.org/t/how-to-tile-a-tensor/13853/3
        """
        init_dim = a.size(dim)
        repeat_idx = [1] * a.dim()
        repeat_idx[dim] = n_tile
        a = a.repeat(*(repeat_idx))
        order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)])).to(a.device)
        return torch.index_select(a, dim, order_index)

    def forward(self, feature_map, z):
        """
        Z is batch_sizexlatent_dim and feature_map is batch_sizexno_channelsxHxW.
        So broadcast Z to batch_sizexlatent_dimxHxW. Behavior is exactly the same as tf.tile (verified)
        """
        if self.use_tile:
            z = torch.unsqueeze(z,2)
            z = self.tile(z, 2, feature_map.shape[self.spatial_axes[0]])
            z = torch.unsqueeze(z,3)
            z = self.tile(z, 3, feature_map.shape[self.spatial_axes[1]])

            #Concatenate the feature map (output of the UNet) and the sample taken from the latent space
            feature_map = torch.cat((feature_map, z), dim=self.channel_axis)
            output = self.layers(feature_map)
            return self.last_layer(output)

In [ ]:
# https://github.com/stefanknegt/Probabilistic-Unet-Pytorch/blob/80152817d90d396a6285cfa24a42c6f255b02ba4/utils.py

def truncated_normal_(tensor, mean=0, std=1):
    size = tensor.shape
    tmp = tensor.new_empty(size + (4,)).normal_()
    valid = (tmp < 2) & (tmp > -2)
    ind = valid.max(-1, keepdim=True)[1]
    tensor.data.copy_(tmp.gather(-1, ind).squeeze(-1))
    tensor.data.mul_(std).add_(mean)

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.ConvTranspose2d:
        nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
        #nn.init.normal_(m.weight, std=0.001)
        #nn.init.normal_(m.bias, std=0.001)
        truncated_normal_(m.bias, mean=0, std=0.001)

## Encoder

In [ ]:
filter_nums = {18: (64, 64, 128, 256, 512),
               34: (64, 64, 128, 256, 512)}


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self,
                 inplanes,
                 planes,
                 stride=1,
                 dilation=1,
                 downsample=None,
                 style='pytorch',
                 with_cp=False,
                 conv_cfg=None,
                 norm_cfg=dict(type='BN')):
        super(BasicBlock, self).__init__()

        self.norm1_name, norm1 = build_norm_layer(norm_cfg, planes, postfix=1)
        self.norm2_name, norm2 = build_norm_layer(norm_cfg, planes, postfix=2)

        self.conv1 = build_conv_layer(
            conv_cfg,
            inplanes,
            planes,
            3,
            stride=stride,
            padding=dilation,
            dilation=dilation,
            bias=False)
        self.add_module(self.norm1_name, norm1)
        self.conv2 = build_conv_layer(
            conv_cfg, planes, planes, 3, padding=1, bias=False)
        self.add_module(self.norm2_name, norm2)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation
        assert not with_cp

    @property
    def norm1(self):
        return getattr(self, self.norm1_name)

    @property
    def norm2(self):
        return getattr(self, self.norm2_name)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.norm2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self,
                 inplanes,
                 planes,
                 stride=1,
                 dilation=1,
                 downsample=None,
                 style='pytorch',
                 with_cp=False,
                 conv_cfg=None,
                 norm_cfg=dict(type='BN')):
        """Bottleneck block for ResNet.
        If style is "pytorch", the stride-two layer is the 3x3 conv layer,
        if it is "caffe", the stride-two layer is the first 1x1 conv layer.
        """
        super(Bottleneck, self).__init__()
        assert style in ['pytorch', 'caffe']

        self.inplanes = inplanes
        self.planes = planes
        self.stride = stride
        self.dilation = dilation
        self.style = style
        self.with_cp = with_cp
        self.conv_cfg = conv_cfg
        self.norm_cfg = norm_cfg

        if self.style == 'pytorch':
            self.conv1_stride = 1
            self.conv2_stride = stride
        else:
            self.conv1_stride = stride
            self.conv2_stride = 1

        self.norm1_name, norm1 = build_norm_layer(norm_cfg, planes, postfix=1)
        self.norm2_name, norm2 = build_norm_layer(norm_cfg, planes, postfix=2)
        self.norm3_name, norm3 = build_norm_layer(
            norm_cfg, planes * self.expansion, postfix=3)

        self.conv1 = build_conv_layer(
            conv_cfg,
            inplanes,
            planes,
            kernel_size=1,
            stride=self.conv1_stride,
            bias=False)
        self.add_module(self.norm1_name, norm1)
        self.conv2 = build_conv_layer(
            conv_cfg,
            planes,
            planes,
            kernel_size=3,
            stride=self.conv2_stride,
            padding=dilation,
            dilation=dilation,
            bias=False)
        self.add_module(self.norm2_name, norm2)
        self.conv3 = build_conv_layer(
            conv_cfg,
            planes,
            planes * self.expansion,
            kernel_size=1,
            bias=False)
        self.add_module(self.norm3_name, norm3)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    @property
    def norm1(self):
        return getattr(self, self.norm1_name)

    @property
    def norm2(self):
        return getattr(self, self.norm2_name)

    @property
    def norm3(self):
        return getattr(self, self.norm3_name)

    def forward(self, x):

        def _inner_forward(x):
            identity = x

            out = self.conv1(x)
            out = self.norm1(out)
            out = self.relu(out)

            out = self.conv2(out)
            out = self.norm2(out)
            out = self.relu(out)

            out = self.conv3(out)
            out = self.norm3(out)

            if self.downsample is not None:
                identity = self.downsample(x)

            out += identity

            return out

        if self.with_cp and x.requires_grad:
            out = cp.checkpoint(_inner_forward, x)
        else:
            out = _inner_forward(x)

        out = self.relu(out)

        return out


def make_res_layer(block,
                   inplanes,
                   planes,
                   blocks,
                   stride=1,
                   dilation=1,
                   style='pytorch',
                   with_cp=False,
                   conv_cfg=None,
                   norm_cfg=dict(type='BN')):
    downsample = None
    if stride != 1 or inplanes != planes * block.expansion:
        downsample = nn.Sequential(
            build_conv_layer(
                conv_cfg,
                inplanes,
                planes * block.expansion,
                kernel_size=1,
                stride=stride,
                bias=False),
            build_norm_layer(norm_cfg, planes * block.expansion)[1],
        )

    layers = []
    layers.append(
        block(
            inplanes=inplanes,
            planes=planes,
            stride=stride,
            dilation=dilation,
            downsample=downsample,
            style=style,
            with_cp=with_cp,
            conv_cfg=conv_cfg,
            norm_cfg=norm_cfg))
    inplanes = planes * block.expansion
    for i in range(1, blocks):
        layers.append(
            block(
                inplanes=inplanes,
                planes=planes,
                stride=1,
                dilation=dilation,
                style=style,
                with_cp=with_cp,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg))

    return nn.Sequential(*layers)


class ResNet(nn.Module):
    """ResNet backbone.
    Args:
        depth (int): Depth of resnet, from {18, 34, 50, 101, 152}.
        in_channels (int): Number of input image channels. Normally 3.
        num_stages (int): Resnet stages, normally 4.
        strides (Sequence[int]): Strides of the first block of each stage.
        dilations (Sequence[int]): Dilation of each stage.
        out_indices (Sequence[int]): Output from which stages.
        style (str): `pytorch` or `caffe`. If set to "pytorch", the stride-two
            layer is the 3x3 conv layer, otherwise the stride-two layer is
            the first 1x1 conv layer.
        frozen_stages (int): Stages to be frozen (stop grad and set eval mode).
            -1 means not freezing any parameters.
        norm_cfg (dict): dictionary to construct and config norm layer.
        norm_eval (bool): Whether to set norm layers to eval mode, namely,
            freeze running stats (mean and var). Note: Effect on Batch Norm
            and its variants only.
        with_cp (bool): Use checkpoint or not. Using checkpoint will save some
            memory while slowing down the training speed.
        zero_init_residual (bool): whether to use zero init for last norm layer
            in resblocks to let them behave as identity.
    Example:
        >>> from openselfsup.models import ResNet
        >>> import torch
        >>> self = ResNet(depth=18)
        >>> self.eval()
        >>> inputs = torch.rand(1, 3, 32, 32)
        >>> level_outputs = self.forward(inputs)
        >>> for level_out in level_outputs:
        ...     print(tuple(level_out.shape))
        (1, 64, 8, 8)
        (1, 128, 4, 4)
        (1, 256, 2, 2)
        (1, 512, 1, 1)
    """

    arch_settings = {
        18: (BasicBlock, (2, 2, 2, 2)),
        34: (BasicBlock, (3, 4, 6, 3)),
        50: (Bottleneck, (3, 4, 6, 3)),
        101: (Bottleneck, (3, 4, 23, 3)),
        152: (Bottleneck, (3, 8, 36, 3))
    }

    def __init__(self,
                 depth,
                 in_channels=3,
                 num_stages=4,
                 strides=(1, 2, 2, 2),
                 dilations=(1, 1, 1, 1),
                 out_indices=(0, 1, 2, 3, 4),
                 style='pytorch',
                 frozen_stages=-1,
                 conv_cfg=None,
                 norm_cfg=dict(type='BN', requires_grad=True),
                 norm_eval=False,
                 with_cp=False,
                 zero_init_residual=False):
        super(ResNet, self).__init__()
        if depth not in self.arch_settings:
            raise KeyError('invalid depth {} for resnet'.format(depth))
        self.depth = depth
        self.num_stages = num_stages
        assert num_stages >= 1 and num_stages <= 4
        self.strides = strides
        self.dilations = dilations
        assert len(strides) == len(dilations) == num_stages
        self.out_indices = out_indices
        assert max(out_indices) < num_stages + 1
        self.style = style
        self.frozen_stages = frozen_stages
        self.conv_cfg = conv_cfg
        self.norm_cfg = norm_cfg
        self.with_cp = with_cp
        self.norm_eval = norm_eval
        self.zero_init_residual = zero_init_residual
        self.block, stage_blocks = self.arch_settings[depth]
        self.stage_blocks = stage_blocks[:num_stages]
        self.inplanes = 64

        self._make_stem_layer(in_channels)

        self.res_layers = []
        for i, num_blocks in enumerate(self.stage_blocks):
            stride = strides[i]
            dilation = dilations[i]
            planes = 64 * 2 ** i
            res_layer = make_res_layer(
                self.block,
                self.inplanes,
                planes,
                num_blocks,
                stride=stride,
                dilation=dilation,
                style=self.style,
                with_cp=with_cp,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg)
            self.inplanes = planes * self.block.expansion
            layer_name = 'layer{}'.format(i + 1)
            self.add_module(layer_name, res_layer)
            self.res_layers.append(layer_name)

        self._freeze_stages()
        self.feat_dims = [64]
        self.feat_dims.extend([self.block.expansion * 64 * 2 ** (i) for i in range(len(self.stage_blocks))])

    @property
    def norm1(self):
        return getattr(self, self.norm1_name)

    @property
    def norm2(self):
        return getattr(self, self.norm2_name)

    def _make_stem_layer(self, in_channels):

        self.conv1 = build_conv_layer(
            self.conv_cfg,
            in_channels,
            64,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False)
        self.norm1_name, norm1 = build_norm_layer(self.norm_cfg, 64, postfix=1)
        self.add_module(self.norm1_name, norm1)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = build_conv_layer(
            self.conv_cfg,
            64,
            64,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False)
        self.norm2_name, norm2 = build_norm_layer(self.norm_cfg, 64, postfix=2)
        self.add_module(self.norm2_name, norm2)

        self.stem = nn.Sequential(self.conv1, self.norm1, self.relu,
                                  self.conv2, self.norm2, self.relu)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def _freeze_stages(self):
        if self.frozen_stages >= 0:
            self.norm1.eval()
            for m in [self.conv1, self.norm1]:
                for param in m.parameters():
                    param.requires_grad = False

        for i in range(1, self.frozen_stages + 1):
            m = getattr(self, 'layer{}'.format(i))
            m.eval()
            for param in m.parameters():
                param.requires_grad = False

    def init_weights(self, pretrained=None):
        if isinstance(pretrained, str):
            logger = get_root_logger()
            load_checkpoint(self, pretrained, strict=False, logger=logger)
        elif pretrained is None:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    kaiming_init(m, mode='fan_in', nonlinearity='relu')
                elif isinstance(m, (_BatchNorm, nn.GroupNorm)):
                    constant_init(m, 1)

            if self.zero_init_residual:
                for m in self.modules():
                    if isinstance(m, Bottleneck):
                        constant_init(m.norm3, 0)
                    elif isinstance(m, BasicBlock):
                        constant_init(m.norm2, 0)
        else:
            raise TypeError('pretrained must be a str or None')

    def forward(self, x):
        outs = []
        x = self.stem(x)  # r50: 64x128x128
        if 0 in self.out_indices:
            outs.append(x)
        x = self.maxpool(x)  # r50: 64x56x56
        for i, layer_name in enumerate(self.res_layers):
            res_layer = getattr(self, layer_name)
            x = res_layer(x)
            if i + 1 in self.out_indices:
                outs.append(x)
        # r50: 1-256x56x56; 2-512x28x28; 3-1024x14x14; 4-2048x7x7
        return outs

    def train(self, mode=True):
        super(ResNet, self).train(mode)
        self._freeze_stages()
        if mode and self.norm_eval:
            for m in self.modules():
                # trick: eval have effect on BatchNorm only
                if isinstance(m, _BatchNorm):
                    m.eval()

## Center

In [ ]:
class MCTrans(nn.Module):
    def __init__(self,
                 d_model=240,
                 nhead=8,
                 d_ffn=1024,
                 dropout=0.1,
                 act="relu",
                 n_points=4,
                 n_levels=3,
                 n_sa_layers=6,
                 in_channles=[64, 64, 128, 256, 512],
                 proj_idxs=(2, 3, 4),

                 ):
        super().__init__()
        self.nhead = nhead
        self.d_model = d_model
        self.n_levels = n_levels

        self.proj_idxs = proj_idxs
        self.projs = nn.ModuleList()
        for idx in self.proj_idxs:
            self.projs.append(ConvModule(in_channles[idx],
                                         d_model,
                                         kernel_size=3,
                                         padding=1,
                                         conv_cfg=dict(type="Conv"),
                                         norm_cfg=dict(type='BN'),
                                         act_cfg=dict(type='ReLU')
                                         ))

        dsa_layer = DSALayer(d_model=d_model,
                             d_ffn=d_ffn,
                             dropout=dropout,
                             activation=act,
                             n_levels=n_levels,
                             n_heads=nhead,
                             n_points=n_points)

        self.dsa = DSA(att_layer=dsa_layer,
                       n_layers=n_sa_layers)

        self.level_embed = nn.Parameter(torch.Tensor(n_levels, d_model))
        self.position_embedding = build_position_encoding(position_embedding="sine", hidden_dim=d_model)
        self._reset_parameters()

    def _reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        for m in self.modules():
            if isinstance(m, MSDeformAttn):
                m._reset_parameters()
        normal_(self.level_embed)

    def get_valid_ratio(self, mask):
        _, H, W = mask.shape
        valid_H = torch.sum(~mask[:, :, 0], 1)
        valid_W = torch.sum(~mask[:, 0, :], 1)
        valid_ratio_h = valid_H.float() / H
        valid_ratio_w = valid_W.float() / W
        valid_ratio = torch.stack([valid_ratio_w, valid_ratio_h], -1)
        return valid_ratio

    def projection(self, feats):
        pos = []
        masks = []
        cnn_feats = []
        tran_feats = []

        for idx, feats in enumerate(feats):
            if idx not in self.proj_idxs:
                cnn_feats.append(feats)
            else:
                n, c, h, w = feats.shape
                mask = torch.zeros((n, h, w)).to(torch.bool).to(feats.device)
                nested_feats = NestedTensor(feats, mask)
                masks.append(mask)
                pos.append(self.position_embedding(nested_feats).to(nested_feats.tensors.dtype))
                tran_feats.append(feats)

        for idx, proj in enumerate(self.projs):
            tran_feats[idx] = proj(tran_feats[idx])

        return cnn_feats, tran_feats, pos, masks

    def forward(self, x):
        # project and prepare for the input
        cnn_feats, trans_feats, pos_embs, masks = self.projection(x)
        # dsa
        features_flatten = []
        mask_flatten = []
        lvl_pos_embed_flatten = []
        feature_shapes = []
        spatial_shapes = []
        for lvl, (feature, mask, pos_embed) in enumerate(zip(trans_feats, masks, pos_embs)):
            bs, c, h, w = feature.shape
            spatial_shapes.append((h, w))
            feature_shapes.append(feature.shape)

            feature = feature.flatten(2).transpose(1, 2)
            mask = mask.flatten(1)
            pos_embed = pos_embed.flatten(2).transpose(1, 2)
            lvl_pos_embed = pos_embed + self.level_embed[lvl].view(1, 1, -1)
            lvl_pos_embed_flatten.append(lvl_pos_embed)

            features_flatten.append(feature)
            mask_flatten.append(mask)

        features_flatten = torch.cat(features_flatten, 1)
        mask_flatten = torch.cat(mask_flatten, 1)
        lvl_pos_embed_flatten = torch.cat(lvl_pos_embed_flatten, 1)
        spatial_shapes = torch.as_tensor(spatial_shapes, dtype=torch.long, device=features_flatten.device)
        level_start_index = torch.cat((spatial_shapes.new_zeros((1,)), spatial_shapes.prod(1).cumsum(0)[:-1]))
        valid_ratios = torch.stack([self.get_valid_ratio(m) for m in masks], 1)

        # self att
        feats = self.dsa(features_flatten,
                         spatial_shapes,
                         level_start_index,
                         valid_ratios,
                         lvl_pos_embed_flatten,
                         mask_flatten)
        # recover
        out = []
        features = feats.split(spatial_shapes.prod(1).tolist(), dim=1)
        for idx, (feats, ori_shape) in enumerate(zip(features, spatial_shapes)):
            out.append(feats.transpose(1, 2).reshape(feature_shapes[idx]))

        cnn_feats.extend(out)
        return cnn_feats

In [ ]:
def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


def _get_activation_fn(activation):
    """Return an activation function given a string"""
    if activation == "relu":
        return F.relu
    if activation == "gelu":
        return F.gelu
    if activation == "glu":
        return F.glu
    raise RuntimeError(F"activation should be relu/gelu, not {activation}.")

def build_position_encoding(position_embedding='sine', hidden_dim=240):
    N_steps = hidden_dim // 2
    if position_embedding in ('v2', 'sine'):
        position_embedding = PositionEmbeddingSine(N_steps, normalize=True)
    elif position_embedding in ('v3', 'learned'):
        position_embedding = PositionEmbeddingLearned(N_steps)
    else:
        raise ValueError(f"not supported {position_embedding}")

    return position_embedding

class NestedTensor(object):
    def __init__(self, tensors, mask: Optional[Tensor]):
        self.tensors = tensors
        self.mask = mask

    def to(self, device, non_blocking=False):
        # type: (Device) -> NestedTensor # noqa
        cast_tensor = self.tensors.to(device, non_blocking=non_blocking)
        mask = self.mask
        if mask is not None:
            assert mask is not None
            cast_mask = mask.to(device, non_blocking=non_blocking)
        else:
            cast_mask = None
        return NestedTensor(cast_tensor, cast_mask)

    def record_stream(self, *args, **kwargs):
        self.tensors.record_stream(*args, **kwargs)
        if self.mask is not None:
            self.mask.record_stream(*args, **kwargs)

    def decompose(self):
        return self.tensors, self.mask

    def __repr__(self):
        return str(self.tensors)

class PositionEmbeddingSine(nn.Module):
    """
    This is a more standard version of the position embedding, very similar to the one
    used by the Attention is all you need paper, generalized to work on images.
    """

    def __init__(self, num_pos_feats=64, temperature=10000, normalize=False, scale=None):
        super().__init__()
        self.num_pos_feats = num_pos_feats
        self.temperature = temperature
        self.normalize = normalize
        if scale is not None and normalize is False:
            raise ValueError("normalize should be True if scale is passed")
        if scale is None:
            scale = 2 * math.pi
        self.scale = scale

    def forward(self, tensor_list: NestedTensor):
        x = tensor_list.tensors
        mask = tensor_list.mask
        assert mask is not None
        not_mask = ~mask
        y_embed = not_mask.cumsum(1, dtype=torch.float32)
        x_embed = not_mask.cumsum(2, dtype=torch.float32)
        if self.normalize:
            eps = 1e-6
            y_embed = (y_embed - 0.5) / (y_embed[:, -1:, :] + eps) * self.scale
            x_embed = (x_embed - 0.5) / (x_embed[:, :, -1:] + eps) * self.scale

        dim_t = torch.arange(self.num_pos_feats, dtype=torch.float32, device=x.device)
        dim_t = self.temperature ** (2 * (dim_t // 3) / self.num_pos_feats)

        pos_x = x_embed[:, :, :, None] / dim_t
        pos_y = y_embed[:, :, :, None] / dim_t
        pos_x = torch.stack((pos_x[:, :, :, 0::2].sin(), pos_x[:, :, :, 1::2].cos()), dim=4).flatten(3)
        pos_y = torch.stack((pos_y[:, :, :, 0::2].sin(), pos_y[:, :, :, 1::2].cos()), dim=4).flatten(3)
        pos = torch.cat((pos_y, pos_x), dim=3).permute(0, 3, 1, 2)
        return pos

In [ ]:
class MSDeformAttnFunction(Function):
    @staticmethod
    def forward(ctx, value, value_spatial_shapes, value_level_start_index, sampling_locations, attention_weights, im2col_step):
        ctx.im2col_step = im2col_step
        output = MSDA.ms_deform_attn_forward(
            value, value_spatial_shapes, value_level_start_index, sampling_locations, attention_weights, ctx.im2col_step)
        ctx.save_for_backward(value, value_spatial_shapes, value_level_start_index, sampling_locations, attention_weights)
        return output

    @staticmethod
    @once_differentiable
    def backward(ctx, grad_output):
        value, value_spatial_shapes, value_level_start_index, sampling_locations, attention_weights = ctx.saved_tensors
        grad_value, grad_sampling_loc, grad_attn_weight = \
            MSDA.ms_deform_attn_backward(
                value, value_spatial_shapes, value_level_start_index, sampling_locations, attention_weights, grad_output, ctx.im2col_step)

        return grad_value, None, None, grad_sampling_loc, grad_attn_weight, None

In [ ]:
def _is_power_of_2(n):
    if (not isinstance(n, int)) or (n < 0):
        raise ValueError("invalid input for _is_power_of_2: {} (type: {})".format(n, type(n)))
    return (n & (n-1) == 0) and n != 0

class MSDeformAttn(nn.Module):
    def __init__(self, d_model=256, n_levels=4, n_heads=8, n_points=4):
        """
        Multi-Scale Deformable Attention Module
        :param d_model      hidden dimension
        :param n_levels     number of feature levels
        :param n_heads      number of attention heads
        :param n_points     number of sampling points per attention head per feature level
        """
        super().__init__()
        if d_model % n_heads != 0:
            raise ValueError('d_model must be divisible by n_heads, but got {} and {}'.format(d_model, n_heads))
        _d_per_head = d_model // n_heads
        # you'd better set _d_per_head to a power of 2 which is more efficient in our CUDA implementation
        if not _is_power_of_2(_d_per_head):
            warnings.warn("You'd better set d_model in MSDeformAttn to make the dimension of each attention head a power of 2 "
                          "which is more efficient in our CUDA implementation.")

        self.im2col_step = 64

        self.d_model = d_model
        self.n_levels = n_levels
        self.n_heads = n_heads
        self.n_points = n_points

        self.sampling_offsets = nn.Linear(d_model, n_heads * n_levels * n_points * 2)
        self.attention_weights = nn.Linear(d_model, n_heads * n_levels * n_points)
        self.value_proj = nn.Linear(d_model, d_model)
        self.output_proj = nn.Linear(d_model, d_model)

        self._reset_parameters()

    def _reset_parameters(self):
        constant_(self.sampling_offsets.weight.data, 0.)
        thetas = torch.arange(self.n_heads, dtype=torch.float32) * (2.0 * math.pi / self.n_heads)
        grid_init = torch.stack([thetas.cos(), thetas.sin()], -1)
        grid_init = (grid_init / grid_init.abs().max(-1, keepdim=True)[0]).view(self.n_heads, 1, 1, 2).repeat(1, self.n_levels, self.n_points, 1)
        for i in range(self.n_points):
            grid_init[:, :, i, :] *= i + 1
        with torch.no_grad():
            self.sampling_offsets.bias = nn.Parameter(grid_init.view(-1))
        constant_(self.attention_weights.weight.data, 0.)
        constant_(self.attention_weights.bias.data, 0.)
        xavier_uniform_(self.value_proj.weight.data)
        constant_(self.value_proj.bias.data, 0.)
        xavier_uniform_(self.output_proj.weight.data)
        constant_(self.output_proj.bias.data, 0.)

    def forward(self, query, reference_points, input_flatten, input_spatial_shapes, input_level_start_index, input_padding_mask=None):
        """
        :param query                       (N, Length_{query}, C)
        :param reference_points            (N, Length_{query}, n_levels, 2), range in [0, 1], top-left (0,0), bottom-right (1, 1), including padding area
                                        or (N, Length_{query}, n_levels, 4), add additional (w, h) to form reference boxes
        :param input_flatten               (N, \sum_{l=0}^{L-1} H_l \cdot W_l, C)
        :param input_spatial_shapes        (n_levels, 2), [(H_0, W_0), (H_1, W_1), ..., (H_{L-1}, W_{L-1})]
        :param input_level_start_index     (n_levels, ), [0, H_0*W_0, H_0*W_0+H_1*W_1, H_0*W_0+H_1*W_1+H_2*W_2, ..., H_0*W_0+H_1*W_1+...+H_{L-1}*W_{L-1}]
        :param input_padding_mask          (N, \sum_{l=0}^{L-1} H_l \cdot W_l), True for padding elements, False for non-padding elements
        :return output                     (N, Length_{query}, C)
        """
        N, Len_q, _ = query.shape
        N, Len_in, _ = input_flatten.shape
        assert (input_spatial_shapes[:, 0] * input_spatial_shapes[:, 1]).sum() == Len_in

        value = self.value_proj(input_flatten)
        if input_padding_mask is not None:
            value = value.masked_fill(input_padding_mask[..., None], float(0))
        value = value.view(N, Len_in, self.n_heads, self.d_model // self.n_heads)
        sampling_offsets = self.sampling_offsets(query).view(N, Len_q, self.n_heads, self.n_levels, self.n_points, 2)
        attention_weights = self.attention_weights(query).view(N, Len_q, self.n_heads, self.n_levels * self.n_points)
        attention_weights = F.softmax(attention_weights, -1).view(N, Len_q, self.n_heads, self.n_levels, self.n_points)
        # N, Len_q, n_heads, n_levels, n_points, 2
        if reference_points.shape[-1] == 2:
            offset_normalizer = torch.stack([input_spatial_shapes[..., 1], input_spatial_shapes[..., 0]], -1)
            sampling_locations = reference_points[:, :, None, :, None, :] \
                                 + sampling_offsets / offset_normalizer[None, None, None, :, None, :]
        elif reference_points.shape[-1] == 4:
            sampling_locations = reference_points[:, :, None, :, None, :2] \
                                 + sampling_offsets / self.n_points * reference_points[:, :, None, :, None, 2:] * 0.5
        else:
            raise ValueError(
                'Last dim of reference_points must be 2 or 4, but get {} instead.'.format(reference_points.shape[-1]))
        output = MSDeformAttnFunction.apply(
            value, input_spatial_shapes, input_level_start_index, sampling_locations, attention_weights, self.im2col_step)
        output = self.output_proj(output)
        return output

In [ ]:
class DSALayer(nn.Module):
    def __init__(self,
                 d_model=256, d_ffn=1024,
                 dropout=0.1, activation="relu",
                 n_levels=4, n_heads=8, n_points=4):
        super().__init__()

        # self attention
        self.self_attn = MSDeformAttn(d_model, n_levels, n_heads, n_points)
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)

        # ffn
        self.linear1 = nn.Linear(d_model, d_ffn)
        self.activation = _get_activation_fn(activation)
        self.dropout2 = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ffn, d_model)
        self.dropout3 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model)

    @staticmethod
    def with_pos_embed(tensor, pos):
        return tensor if pos is None else tensor + pos

    def forward_ffn(self, src):
        src2 = self.linear2(self.dropout2(self.activation(self.linear1(src))))
        src = src + self.dropout3(src2)
        src = self.norm2(src)
        return src

    def forward(self, src, pos, reference_points, spatial_shapes, level_start_index, padding_mask=None):
        # self attention
        src2 = self.self_attn(self.with_pos_embed(src, pos),
                              reference_points,
                              src,
                              spatial_shapes,
                              level_start_index,
                              padding_mask)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        # ffn
        src = self.forward_ffn(src)

        return src


class DSA(nn.Module):
    def __init__(self, att_layer, n_layers):
        super().__init__()
        self.layers = _get_clones(att_layer, n_layers)

    @staticmethod
    def get_reference_points(spatial_shapes, valid_ratios, device):
        reference_points_list = []
        for lvl, (H_, W_) in enumerate(spatial_shapes):
            ref_y, ref_x = torch.meshgrid(torch.linspace(0.5, H_ - 0.5, H_, dtype=torch.float32, device=device),
                                          torch.linspace(0.5, W_ - 0.5, W_, dtype=torch.float32, device=device))
            ref_y = ref_y.reshape(-1)[None] / (valid_ratios[:, None, lvl, 1] * H_)
            ref_x = ref_x.reshape(-1)[None] / (valid_ratios[:, None, lvl, 0] * W_)
            ref = torch.stack((ref_x, ref_y), -1)
            reference_points_list.append(ref)
        reference_points = torch.cat(reference_points_list, 1)
        reference_points = reference_points[:, :, None] * valid_ratios[:, None]
        return reference_points

    def forward(self, src, spatial_shapes, level_start_index, valid_ratios, pos=None, padding_mask=None):
        output = src
        reference_points = self.get_reference_points(spatial_shapes, valid_ratios, device=src.device)
        for _, layer in enumerate(self.layers):
            output = layer(output, pos, reference_points, spatial_shapes, level_start_index, padding_mask)
        return output


class CALayer(nn.Module):
    def __init__(self, d_model=256, d_ffn=1024,
                 dropout=0.1, activation="relu", n_heads=8):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)

        # self attention
        self.cross_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model)

        # ffn
        self.linear1 = nn.Linear(d_model, d_ffn)
        self.activation = _get_activation_fn(activation)
        self.dropout3 = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ffn, d_model)
        self.dropout4 = nn.Dropout(dropout)
        self.norm3 = nn.LayerNorm(d_model)

    @staticmethod
    def with_pos_embed(tensor, pos):
        return tensor if pos is None else tensor + pos

    def forward_ffn(self, tgt):
        tgt2 = self.linear2(self.dropout3(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout4(tgt2)
        tgt = self.norm3(tgt)
        return tgt

    def forward(self, tgt, src):
        # self attention
        q = k = tgt
        tgt2 = self.self_attn(q.transpose(0, 1), k.transpose(0, 1), tgt.transpose(0, 1))[0].transpose(0, 1)
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        # cross attention
        tgt2 = self.cross_attn(tgt, src, src)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        # ffn
        tgt = self.forward_ffn(tgt)

        return tgt


class CA(nn.Module):
    def __init__(self, att_layer, n_layers, n_category=2, d_model=256):
        super().__init__()
        self.layers = _get_clones(att_layer, n_layers)
        self.proxy_embed = nn.Parameter(torch.zeros(1, n_category, d_model))

    def forward(self, src):
        query = None
        B = src.shape[0]
        for idx, layer in enumerate(self.layers):
            if idx == 0:
                query = self.proxy_embed.expand(B, -1, -1)
            else:
                query += self.proxy_embed.expand(B, -1, -1)
            query = layer(query, src)
        return query

## Decoder

In [ ]:
def conv_bn_relu(in_channels, out_channels, kernel_size=3, padding=1, stride=1):
    return nn.Sequential(nn.Conv2d(in_channels,
                                   out_channels,
                                   kernel_size=kernel_size,
                                   padding=padding,
                                   stride=stride),
                         nn.BatchNorm2d(out_channels),
                         nn.ReLU(inplace=True))

In [ ]:
class AttBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttBlock, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(in_channels=F_g,
                      out_channels=F_int,
                      kernel_size=1,
                      stride=1,
                      padding=0,
                      bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(in_channels=F_l,
                      out_channels=F_int,
                      kernel_size=1,
                      stride=1,
                      padding=0,
                      bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(in_channels=F_int,
                      out_channels=1,
                      kernel_size=1,
                      stride=1,
                      padding=0,
                      bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)

        return x * psi


class DecBlock(nn.Module):
    def __init__(
            self,
            in_channels,
            skip_channels,
            out_channels,
            attention=False
    ):
        super().__init__()
        self.conv1 = conv_bn_relu(in_channels=in_channels + skip_channels,
                                  out_channels=out_channels)

        self.conv2 = conv_bn_relu(in_channels=out_channels,
                                  out_channels=out_channels)

        self.up = nn.Upsample(scale_factor=2,
                              mode='bilinear',
                              align_corners=True)

        if attention:
            self.att = AttBlock(F_g=in_channels, F_l=skip_channels, F_int=in_channels)

    def forward(self, x, skip=None):
        x = self.up(x)
        if skip is not None:
            if hasattr(self, "att"):
                skip = self.att(g=x, x=skip)
            x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

class UNetDecoder(nn.Module):
    def __init__(
            self,
            in_channels,
            att=False
    ):
        super().__init__()
        self.decoders = nn.ModuleList()
        in_channels = in_channels[::-1]
        skip_channels = in_channels[1:]
        for in_c, skip_c in zip(in_channels, skip_channels):
            self.decoders.append(DecBlock(in_c, skip_c, skip_c, att))

    def forward(self, features):
        features = features[::-1]
        x = features[0]
        skips = features[1:]

        for i, layer in enumerate(self.decoders):
            x = layer(x, skips[i])

        return x

    def init_weights(self):
        pass

## Head

In [ ]:
class MCTransAuxHead(nn.Module):
    def __init__(self,
                 d_model=240,
                 d_ffn=1024,
                 dropout=0.1,
                 act="relu",
                 n_head=8,
                 n_layers=4,
                 num_classes=1,
                 in_channles=[64, 64, 128, 256, 512],
                 proj_idxs=(2, 3, 4),
                 losses=None
                 ):
        super(MCTransAuxHead, self).__init__()
        self.in_channles = in_channles
        self.proj_idxs = proj_idxs

        ca_layer = CALayer(d_model=d_model,
                           d_ffn=d_ffn,
                           dropout=dropout,
                           activation=act,
                           n_heads=n_head)

        self.ca = CA(att_layer=ca_layer,
                     n_layers=n_layers,
                     n_category=num_classes,
                     d_model=d_model)

        self.head = nn.Sequential(nn.Linear(num_classes*d_model, d_model),
                                  nn.Linear(d_model, num_classes))

    def forward(self, inputs):
        # flatten
        inputs = [inputs[idx] for idx in self.proj_idxs]
        inputs_flatten = [item.flatten(2).transpose(1, 2) for item in inputs]
        inputs_flatten = torch.cat(inputs_flatten, 1)
        # ca
        outputs = self.ca(inputs_flatten)
        logits = self.head(outputs.flatten(1))
       
        return logits

    def init_weights(self):
        normal_init(self.head, mean=0, std=0.01)

## Losses

In [ ]:
class BinaryFocalLoss(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
        Focal_Loss= -1*alpha*(1-pt)*log(pt)
    :param alpha: (tensor) 3D or 4D the scalar factor for this criterion
    :param gamma: (float,double) gamma > 0 reduces the relative loss for well-classified examples (p>0.5) putting more
                    focus on hard misclassified example
    :param reduction: `none`|`mean`|`sum`
    :param **kwargs
        balance_index: (int) balance class index, should be specific when alpha is float
    """

    def __init__(self, alpha=3, gamma=2, ignore_index=None, reduction='mean', **kwargs):
        super(BinaryFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smooth = 1e-6  # set '1e-4' when train with FP16
        self.ignore_index = ignore_index
        self.reduction = reduction

        assert self.reduction in ['none', 'mean', 'sum']

        # if self.alpha is None:
        #     self.alpha = torch.ones(2)
        # elif isinstance(self.alpha, (list, np.ndarray)):
        #     self.alpha = np.asarray(self.alpha)
        #     self.alpha = np.reshape(self.alpha, (2))
        #     assert self.alpha.shape[0] == 2, \
        #         'the `alpha` shape is not match the number of class'
        # elif isinstance(self.alpha, (float, int)):
        #     self.alpha = np.asarray([self.alpha, 1.0 - self.alpha], dtype=np.float).view(2)

        # else:
        #     raise TypeError('{} not supported'.format(type(self.alpha)))

    def forward(self, output, target):
        prob = torch.sigmoid(output)
        prob = torch.clamp(prob, self.smooth, 1.0 - self.smooth)

        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = (target != self.ignore_index).float()

        pos_mask = (target == 1).float()
        neg_mask = (target == 0).float()
        if valid_mask is not None:
            pos_mask = pos_mask * valid_mask
            neg_mask = neg_mask * valid_mask

        pos_weight = (pos_mask * torch.pow(1 - prob, self.gamma)).detach()
        pos_loss = -pos_weight * torch.log(prob)  # / (torch.sum(pos_weight) + 1e-4)

        neg_weight = (neg_mask * torch.pow(prob, self.gamma)).detach()
        neg_loss = -self.alpha * neg_weight * F.logsigmoid(-output)  # / (torch.sum(neg_weight) + 1e-4)
        loss = pos_loss + neg_loss
        loss = loss.mean()
        return loss

def reduce_loss(loss, reduction):
    """Reduce loss as specified.
    Args:
        loss (Tensor): Elementwise loss tensor.
        reduction (str): Options are "none", "mean" and "sum".
    Return:
        Tensor: Reduced loss tensor.
    """
    reduction_enum = F._Reduction.get_enum(reduction)
    # none: 0, elementwise_mean:1, sum: 2
    if reduction_enum == 0:
        return loss
    elif reduction_enum == 1:
        return loss.mean()
    elif reduction_enum == 2:
        return loss.sum()


def weight_reduce_loss(loss, weight=None, reduction='mean', avg_factor=None):
    """Apply element-wise weight and reduce loss.
    Args:
        loss (Tensor): Element-wise loss.
        weight (Tensor): Element-wise weights.
        reduction (str): Same as built-in losses of PyTorch.
        avg_factor (float): Avarage factor when computing the mean of losses.
    Returns:
        Tensor: Processed loss values.
    """
    # if weight is specified, apply element-wise weight
    if weight is not None:
        loss = loss * weight

    # if avg_factor is not specified, just reduce the loss
    if avg_factor is None:
        loss = reduce_loss(loss, reduction)
    else:
        # if reduction is mean, then average the loss by avg_factor
        if reduction == 'mean':
            loss = loss.sum() / avg_factor
        # if reduction is 'none', then do nothing, otherwise raise an error
        elif reduction != 'none':
            raise ValueError('avg_factor can not be used with reduction="sum"')
    return loss


def cross_entropy(pred, label, weight=None, reduction='mean', avg_factor=None):
    """Calculate the CrossEntropy loss.
    Args:
        pred (torch.Tensor): The prediction with shape (N, C), C is the number
            of classes.
        label (torch.Tensor): The gt label of the prediction.
        weight (torch.Tensor, optional): Sample-wise loss weight.
        reduction (str): The method used to reduce the loss.
        avg_factor (int, optional): Average factor that is used to average
            the loss. Defaults to None.
    Returns:
        torch.Tensor: The calculated loss
    """
    # element-wise losses
    loss = F.cross_entropy(pred, label, reduction='none')

    # apply weights and do the reduction
    if weight is not None:
        weight = weight.float()
    loss = weight_reduce_loss(
        loss, weight=weight, reduction=reduction, avg_factor=avg_factor)

    return loss

def binary_cross_entropy(pred,
                         label,
                         weight=None,
                         reduction='mean',
                         avg_factor=None):
    """Calculate the binary CrossEntropy loss with logits.
    Args:
        pred (torch.Tensor): The prediction with shape (N, *).
        label (torch.Tensor): The gt label with shape (N, *).
        weight (torch.Tensor, optional): Element-wise weight of loss with shape
             (N, ). Defaults to None.
        reduction (str): The method used to reduce the loss.
            Options are "none", "mean" and "sum". If reduction is 'none' , loss
             is same shape as pred and label. Defaults to 'mean'.
        avg_factor (int, optional): Average factor that is used to average
            the loss. Defaults to None.
    Returns:
        torch.Tensor: The calculated loss
    """
    # assert pred.dim() == label.dim()

    loss = F.binary_cross_entropy_with_logits(pred, label, reduction='none')

    # apply weights and do the reduction
    if weight is not None:
        assert weight.dim() == 1
        weight = weight.float()
        if pred.dim() > 1:
            weight = weight.reshape(-1, 1)
    loss = weight_reduce_loss(
        loss, weight=weight, reduction=reduction, avg_factor=avg_factor)
    return loss

class CrossEntropyLoss(nn.Module):
    """Cross entropy loss.
    Args:
        use_sigmoid (bool): Whether the prediction uses sigmoid
            of softmax. Defaults to False.
        use_soft (bool): Whether to use the soft version of CrossEntropyLoss.
            Defaults to False.
        reduction (str): The method used to reduce the loss.
            Options are "none", "mean" and "sum". Defaults to 'mean'.
        loss_weight (float):  Weight of the loss. Defaults to 1.0.
    """

    def __init__(self,
                 sigmoid=False,
                 softmax=False,
                 reduction='mean',
                 loss_weight=1.0):
        super(CrossEntropyLoss, self).__init__()
        self.use_sigmoid = sigmoid
        self.use_soft = softmax
        assert not (
                self.use_soft and self.use_sigmoid
        ), 'use_sigmoid and use_soft could not be set simultaneously'

        self.reduction = reduction
        self.loss_weight = loss_weight

        if self.use_sigmoid:
            self.cls_criterion = binary_cross_entropy
        elif self.use_soft:
            self.cls_criterion = soft_cross_entropy
        else:
            self.cls_criterion = cross_entropy

    def forward(self,
                cls_score,
                label,
                weight=None,
                avg_factor=None,
                reduction_override=None,
                **kwargs):
        assert reduction_override in (None, 'none', 'mean', 'sum')
        reduction = (
            reduction_override if reduction_override else self.reduction)

        n_pred_ch, n_target_ch = cls_score.shape[1], label.shape[1]
        if n_pred_ch == n_target_ch:
            label = torch.argmax(label, dim=1)
        else:
            label = torch.squeeze(label, dim=1)
        label = label.long()

        loss_cls = self.loss_weight * self.cls_criterion(
            cls_score,
            label,
            weight,
            reduction=reduction,
            avg_factor=avg_factor,
            **kwargs)
        return loss_cls


class MCTransAuxLoss(CrossEntropyLoss):
    def __init__(self,**kwargs):
        super(MCTransAuxLoss, self).__init__(**kwargs)

    def forward(self,
                cls_score,
                label,
                weight=None,
                avg_factor=None,
                reduction_override=None,
                **kwargs):
        assert reduction_override in (None, 'none', 'mean', 'sum')
        #To one hot
        num_classes = cls_score.shape[1]
        one_hot_list = []

        for l in label:
            one_hot_list.append(self.one_hot(torch.unique(l), num_classes=num_classes).sum(dim=0))
        label = torch.stack(one_hot_list) - 1

        reduction = (
            reduction_override if reduction_override else self.reduction)
        
        loss_cls = self.loss_weight * self.cls_criterion(
            cls_score,
            label,
            weight,
            reduction=reduction,
            avg_factor=avg_factor,
            **kwargs)

        return loss_cls

    def one_hot(self, input, num_classes, dtype=torch.float):
        assert input.dim() > 0, "input should have dim of 1 or more."

        # if 1D, add singelton dim at the end
        if input.dim() == 1:
            input = input.view(-1, 1)

        sh = list(input.shape)

        assert sh[1] == 1, "labels should have a channel with length equals to one."
        sh[1] = num_classes

        o = torch.zeros(size=sh, dtype=dtype, device=input.device)
        labels = o.scatter_(dim=0, index=input.long(), value=1)
        return labels

class VolumeLoss(nn.Module):
    def __init__(self, threshold=0.25):
        super(VolumeLoss, self).__init__()
        self.threshold = threshold

    def forward(self, output, target):
        output[output > self.threshold] = 1.0
        output[output <= self.threshold] = 0.0

        vol_output = output.sum()/1000
        target_output = target.sum()/1000
        vol_loss = torch.abs(target_output-vol_output)
        return vol_loss

class TotalLoss(nn.Module):
    def __init__(self, alpha=3, gamma=2, threshold=0.25):
        super(TotalLoss, self).__init__()
        self.focal_loss = BinaryFocalLoss(alpha=alpha, gamma=gamma)
        self.volume_loss = VolumeLoss(threshold=threshold) 
        self.aux_loss = MCTransAuxLoss(sigmoid=True, loss_weight=0.1)

    def forward(self, output, logits, target):
        fl = self.focal_loss(output, target)
        vl = self.volume_loss(output, target)
        al = self.aux_loss(logits, target)

        total_loss = fl + al #+ vl
        return fl, total_loss

# 4.3. K-Fold (pytorch model)

## Training

In [ ]:
model_name = 'MCTrans'
# ADNI, Challenge
train = 'Challenge'
amount = 2
dsc_list = []
kf = KFold(n_splits=amount)
fold_img, fold_seg = get_dataset(train)

for k, (train_idx, test_idx) in enumerate(kf.split(fold_img)):
    num = 0 if k == 0 else 10 
    if k == 0:
        # Training data
        if train == 'ADNI':
            img_temp = np.concatenate([fold_img[train_idx[0]], fold_img[train_idx[1]]])
            seg_temp = np.concatenate([fold_seg[train_idx[0]], fold_seg[train_idx[1]]])
            img_train = np.concatenate(img_temp[:24])
            seg_train = np.concatenate(seg_temp[:24])
            img_val = np.concatenate(img_temp[-6:])
            seg_val = np.concatenate(seg_temp[-6:])
        else:
            img_temp = [i[num:num+10] for i in fold_img]
            seg_temp = [i[num:num+10] for i in fold_seg]
            img_train = np.concatenate([np.concatenate(i[:-2]) for i in img_temp])
            seg_train = np.concatenate([np.concatenate(i[:-2]) for i in seg_temp])
            img_val = np.concatenate([np.concatenate(i[-2:]) for i in img_temp])
            seg_val = np.concatenate([np.concatenate(i[-2:]) for i in seg_temp])

        del img_temp, seg_temp
        img_train, seg_train = check_data(img_train, seg_train)
        img_val, seg_val = check_data(img_val, seg_val)
        data_augmentation(img_train, seg_train)
        
        img_train = img_train.transpose(0,3,1,2); seg_train = seg_train.transpose(0,3,1,2) #Full
        img_val = img_val.transpose(0,3,1,2); seg_val = seg_val.transpose(0,3,1,2) #Full

        img_train = torch.Tensor(img_train); seg_train = torch.Tensor(seg_train)
        img_val = torch.Tensor(img_val); seg_val = torch.Tensor(seg_val)

        trainset = torch.utils.data.TensorDataset(img_train,seg_train)
        valset = torch.utils.data.TensorDataset(img_val,seg_val)
        
        trainset = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
        valset = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                            shuffle=False, num_workers=2)
        
        del img_train, seg_train, img_val, seg_val
        
        logdir = path+'logdir/{}_{}_fold{}_2_{}_novl_n_points_6'.format(model_name,uniq,k+1,train)
        temp = logdir.split('/')[-1]
        # Becareful with these 4 lines!
        ## Will delete existing folders and files
        if len(glob(path+temp)) >= 1: shutil.rmtree(path+temp)
        ## Will create folders and files
        if len(glob(path+temp)) == 0: os.mkdir(path+temp)

        md = pick_model(model_name)
        if torch.cuda.is_available():
            md = md.cuda()
        criterion = TotalLoss(gamma=gamma, alpha=alpha, threshold=threshold)
        optimizer = torch.optim.Adam(md.parameters(), lr=lr)
        min_valid_loss = np.inf

        for i in range(epoch):
            train_loss, train_fl = 0.0, 0.0
            md.train()     # Optional when not using Model Specific layer
            for data, labels in trainset:
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                optimizer.zero_grad()
                target, logits = md(data, labels)
                
                kl = beta * torch.mean(md.kl_divergence()) if lr_latent is not None else 0
                fl, loss = criterion(target, logits, labels)
                loss = loss + kl
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                train_fl += fl.item()
            
            valid_loss, valid_fl = 0.0, 0.0
            md.eval()     # Optional when not using Model Specific layer
            for data, labels in valset:
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                target, logits = md.sampling(data, labels)
                kl = beta * torch.mean(md.kl_divergence()) if lr_latent is not None else 0
                fl, loss = criterion(target, logits, labels)
                loss = loss + kl
                valid_fl = fl.item() * data.size(0)
                valid_loss = loss.item() * data.size(0)

            print(f'Epoch {i+1} \t Focal Loss: {train_fl / len(trainset)} \t Total Loss: {train_loss / len(trainset)}', end='\t -')
            print(f'Val Focal Loss: {valid_fl / len(valset)} \t Val Total Loss: {valid_loss / len(valset)}')
            if min_valid_loss > valid_fl:
                min_valid_loss = valid_fl
                # Saving State Dict
                torch.save(md.state_dict(), path+temp+"/checkpoint-best.pth")

#     # Testing
    # md.load_state_dict(torch.load(path+temp+"/checkpoint-best.pth"))

#     ## ADNI Testing
#     if train == 'ADNI':
#         dsc_patient = []
#         paths = glob(path_dataset+'ADNI/{}/*'.format('fold'+str(test_idx[0]+1)))
#         paths = paths + glob(path_dataset+'ADNI/{}/*'.format('fold'+str(test_idx[1]+1)))
#         paths.sort(key=natural_keys)

#         img_test = np.concatenate([fold_img[test_idx[0]], fold_img[test_idx[1]]])
#         seg_test = np.concatenate([fold_seg[test_idx[0]], fold_seg[test_idx[1]]])

#         for idx_p, (img, seg) in enumerate(zip(img_test, seg_test)):
#             img = img.transpose(0,3,1,2)
#             seg = seg.transpose(0,3,1,2)
#             img = torch.Tensor(img); seg = torch.Tensor(seg)

#             testset = torch.utils.data.TensorDataset(img,seg)
#             testset = torch.utils.data.DataLoader(testset, batch_size=batch_size, 
#                                                   shuffle=False, num_workers=2)
            
#             y_pred_list = []
#             y_ss_list, y_sy_list = [], []
#             for img_, seg_ in testset:
#                 if torch.cuda.is_available():
#                     img_, seg_ = img_.cuda(), seg_.cuda()

#                 if lr_latent is not None:
#                     y_gen = []
#                     for x in range(generate_pred):
#                         if model_name == 'Prob_MCTrans':
#                             y_pred,_ = md.sampling(img_)
#                             y_pred = y_pred.cpu().data.numpy()
#                         else:
#                             y_pred = md.predict(img_)
#                         y_gen.append(y_pred)

#                     y_gen = np.array(y_gen)
#                     y_ss = ambiguity_map(y_gen)
#                     y_sy = ambiguity_map(y_gen, seg_.cpu().data.numpy())
#                     y_pred = np.average(y_gen, axis=0)

#                 else:
#                     if model_name == 'MCTrans':
#                         y_pred,_ = md(img_, seg_)
#                         y_pred = y_pred.cpu().data.numpy()
#                     else:
#                         y_pred = md.predict(img_)
#                     y_ss = None; y_sy = None
                    
#                 y_pred[y_pred > threshold] = 1
#                 y_pred[y_pred <= threshold] = 0
#                 y_pred_list.append(y_pred)
#                 if lr_latent is not None:
#                     y_ss_list.append(y_ss)
#                     y_sy_list.append(y_sy)

#             y_pred = np.concatenate(y_pred_list)
#             y_pred = y_pred.transpose(0,2,3,1)
#             if lr_latent is not None:
#                 y_ss = np.concatenate(y_ss_list)
#                 y_sy = np.concatenate(y_sy_list)
#                 y_ss = y_ss.transpose(0,2,3,1)
#                 y_sy = y_sy.transpose(0,2,3,1)

#             # img = img.cpu().data.numpy().transpose(0,2,3,1)
#             seg = seg.cpu().data.numpy().transpose(0,2,3,1)
#             dsc = dice_coef(seg, y_pred).numpy()
#             dsc_patient.append(dsc)
            
#             patient_id = paths[idx_p].split('/')[-1]
#             fold_id = paths[idx_p].split('/')[-2]
#             file_in = '{}/{}_wmh.nii.gz'.format(paths[idx_p], patient_id)
#             file_out = result_path.format('ADNI', fold_id, patient_id)
#             if len(glob(file_out)) == 0: os.makedirs(file_out)
#             file_out_pred = file_out+'/{}_wmh_{}.nii.gz'.format(patient_id, model_name)
#             save_wmh(y_pred, file_in, file_out_pred)
#             # show_img(img, seg, y_pred, 18)

#             # Ambiguity maps
#             if y_ss is not None and y_sy is not None:
#                 file_out_1 = file_out+'/{}_am_ss_{}.nii.gz'.format(patient_id, model_name)
#                 file_out_2 = file_out+'/{}_am_sy_{}.nii.gz'.format(patient_id, model_name)
#                 save_wmh(y_ss, file_in, file_out_1)
#                 save_wmh(y_sy, file_in, file_out_2)

#     ## Challenge Testing
#     else:
#         dsc_patient = []
#         for iter_,inst in enumerate(['Singapore', 'GE3T', 'Utrecht']):
#             dsc_temp = []
#             paths = glob(path_dataset+'Challenge/{}/*'.format(inst))
#             paths.sort(key=natural_keys)
#             paths = paths[10-num:20-num]
            
#             img_test = np.array(fold_img[iter_][10-num:20-num], dtype='float32')
#             seg_test = np.array(fold_seg[iter_][10-num:20-num], dtype='float32')

#             for idx_p, (img, seg) in enumerate(zip(img_test, seg_test)):
#                 img = img.transpose(0,3,1,2)
#                 seg = seg.transpose(0,3,1,2)
#                 img = torch.Tensor(img); seg = torch.Tensor(seg)

#                 testset = torch.utils.data.TensorDataset(img,seg)
#                 testset = torch.utils.data.DataLoader(testset, batch_size=batch_size, 
#                                                     shuffle=False, num_workers=2)
                
#                 y_pred_list = []
#                 y_ss_list, y_sy_list = [], []
#                 for img_, seg_ in testset:
#                     if torch.cuda.is_available():
#                         img_, seg_ = img_.cuda(), seg_.cuda()

#                     if lr_latent is not None:
#                         y_gen = []
#                         for x in range(generate_pred):
#                             if model_name == 'Prob_MCTrans':
#                                 y_pred,_ = md.sampling(img_)
#                                 y_pred = y_pred.cpu().data.numpy()
#                             else:
#                                 y_pred = md.predict(img_)
#                             y_gen.append(y_pred)

#                         y_gen = np.array(y_gen)
#                         y_ss = ambiguity_map(y_gen)
#                         y_sy = ambiguity_map(y_gen, seg_.cpu().data.numpy())
#                         y_pred = np.average(y_gen, axis=0)

#                     else:
#                         if model_name == 'MCTrans':
#                             y_pred,_ = md(img_, seg_)
#                             y_pred = y_pred.cpu().data.numpy()
#                         else:
#                             y_pred = md.predict(img_)
#                         y_ss = None; y_sy = None
                        
#                     y_pred[y_pred > threshold] = 1
#                     y_pred[y_pred <= threshold] = 0
#                     y_pred_list.append(y_pred)
#                     if lr_latent is not None:
#                         y_ss_list.append(y_ss)
#                         y_sy_list.append(y_sy)

#                 y_pred = np.concatenate(y_pred_list)
#                 y_pred = y_pred.transpose(0,2,3,1)
#                 if lr_latent is not None:
#                     y_ss = np.concatenate(y_ss_list)
#                     y_sy = np.concatenate(y_sy_list)
#                     y_ss = y_ss.transpose(0,2,3,1)
#                     y_sy = y_sy.transpose(0,2,3,1)

#                 # img = img.cpu().data.numpy().transpose(0,2,3,1)
#                 seg = seg.cpu().data.numpy().transpose(0,2,3,1)
#                 dsc = dice_coef(seg, y_pred).numpy()
#                 dsc_temp.append(dsc)
                
#                 patient_id = paths[idx_p].split('/')[-1]
#                 file_in = '{}/{}_wmh.nii.gz'.format(paths[idx_p], patient_id)
#                 file_out = result_path.format('Challenge', inst, patient_id)
#                 if len(glob(file_out)) == 0: os.makedirs(file_out)
#                 file_out_pred = file_out+'/{}_wmh_{}.nii.gz'.format(patient_id, model_name)
#                 save_wmh_challenge(y_pred, file_in, file_out_pred, name=inst)
#                 # show_img(img, seg, y_pred, 28)

#                 # Ambiguity maps
#                 if y_ss is not None and y_sy is not None:
#                     file_out_1 = file_out+'/{}_am_ss_{}.nii.gz'.format(patient_id, model_name)
#                     file_out_2 = file_out+'/{}_am_sy_{}.nii.gz'.format(patient_id, model_name)
#                     save_wmh_challenge(y_ss, file_in, file_out_1, name=inst)
#                     save_wmh_challenge(y_sy, file_in, file_out_2, name=inst)

#             dsc_patient.append(dsc_temp)
#     dsc_list.append(dsc_patient)
#     print('Average Dice Coefficient on fold-{} : {:.4f}'.format(k+1, np.average(dsc_patient)))
# print('Average Dice Coefficient : {:.4f}'.format(np.average(dsc_list)))

Epoch 1 	 Focal Loss: 0.015879649142550453 	 Total Loss: 0.08726239156165569	 -Val Focal Loss: 3.4807965026370115e-05 	 Val Total Loss: 0.0016523749700614384
Epoch 2 	 Focal Loss: 0.002611007558587242 	 Total Loss: 0.0728116759748982	 -Val Focal Loss: 1.288007728622428e-05 	 Val Total Loss: 0.0019154870084353856
Epoch 3 	 Focal Loss: 0.001662268081025418 	 Total Loss: 0.07163788598004005	 -Val Focal Loss: 9.386359181787287e-05 	 Val Total Loss: 0.001549223278250013
Epoch 4 	 Focal Loss: 0.0015154593956984224 	 Total Loss: 0.07123752686081172	 -Val Focal Loss: 2.617793091173683e-06 	 Val Total Loss: 0.001996400100844247
Epoch 5 	 Focal Loss: 0.0013339648019792948 	 Total Loss: 0.07089892400951266	 -Val Focal Loss: 2.2051533700765245e-06 	 Val Total Loss: 0.0019582054444721767
Epoch 6 	 Focal Loss: 0.0011802909323856026 	 Total Loss: 0.07072532613822025	 -Val Focal Loss: 3.878370625898242e-06 	 Val Total Loss: 0.0018223941326141358
Epoch 7 	 Focal Loss: 0.00117163595446367 	 Total Loss: 

##Testing

In [ ]:
model_name = 'MCTrans'
# ADNI, Challenge
train = 'Challenge'
amount = 2
dsc_list = []
kf = KFold(n_splits=amount)
fold_img, fold_seg = get_dataset(train)

for k, (train_idx, test_idx) in enumerate(kf.split(fold_img)):
    num = 0 if k == 0 else 10 
    # # Training data 
    # if train == 'ADNI':
    #     img_temp = np.concatenate([fold_img[train_idx[0]], fold_img[train_idx[1]]])
    #     seg_temp = np.concatenate([fold_seg[train_idx[0]], fold_seg[train_idx[1]]])
    #     img_train = np.concatenate(img_temp[:24])
    #     seg_train = np.concatenate(seg_temp[:24])
    #     img_val = np.concatenate(img_temp[-6:])
    #     seg_val = np.concatenate(seg_temp[-6:])
    # else:
    #     img_temp = [i[num:num+10] for i in fold_img]
    #     seg_temp = [i[num:num+10] for i in fold_seg]
    #     img_train = np.concatenate([np.concatenate(i[:-2]) for i in img_temp])
    #     seg_train = np.concatenate([np.concatenate(i[:-2]) for i in seg_temp])
    #     img_val = np.concatenate([np.concatenate(i[-2:]) for i in img_temp])
    #     seg_val = np.concatenate([np.concatenate(i[-2:]) for i in seg_temp])

    # del img_temp, seg_temp
    # img_train, seg_train = check_data(img_train, seg_train)
    # img_val, seg_val = check_data(img_val, seg_val)
    # data_augmentation(img_train, seg_train)
    
    # img_train = img_train.transpose(0,3,1,2); seg_train = seg_train.transpose(0,3,1,2) #Full
    # img_val = img_val.transpose(0,3,1,2); seg_val = seg_val.transpose(0,3,1,2) #Full

    # img_train = torch.Tensor(img_train); seg_train = torch.Tensor(seg_train)
    # img_val = torch.Tensor(img_val); seg_val = torch.Tensor(seg_val)

    # trainset = torch.utils.data.TensorDataset(img_train,seg_train)
    # valset = torch.utils.data.TensorDataset(img_val,seg_val)
    
    # trainset = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
    #                                        shuffle=True, num_workers=2, drop_last=True)
    # valset = torch.utils.data.DataLoader(valset, batch_size=batch_size,
    #                                      shuffle=False, num_workers=2, drop_last=True)
    
    # del img_train, seg_train, img_val, seg_val
    
    logdir = path+'logdir/{}_{}_fold{}_2_{}_novl_n_points_6'.format(model_name,uniq,k+1,train)
    temp = logdir.split('/')[-1]
    # # Becareful with these 4 lines!
    # ## Will delete existing folders and files
    # if len(glob(path+temp)) >= 1: shutil.rmtree(path+temp)
    # ## Will create folders and files
    # if len(glob(path+temp)) == 0: os.mkdir(path+temp)

    md = pick_model(model_name)
    if torch.cuda.is_available():
        md = md.cuda()
    # criterion = TotalLoss(gamma=gamma, alpha=alpha, threshold=threshold)
    # optimizer = torch.optim.Adam(md.parameters(), lr=lr)
    # min_valid_loss = np.inf

    # for i in range(epoch):
    #     train_loss, train_fl = 0.0, 0.0
    #     md.train()     # Optional when not using Model Specific layer
    #     for data, labels in trainset:
    #         if torch.cuda.is_available():
    #             data, labels = data.cuda(), labels.cuda()

    #         optimizer.zero_grad()
    #         target, logits = md(data, labels)
            
    #         kl = beta * torch.mean(md.kl_divergence()) if lr_latent is not None else 0
    #         fl, loss = criterion(target, logits, labels)
    #         loss = loss + kl
    #         loss.backward()
    #         optimizer.step()
    #         train_loss += loss.item()
    #         train_fl += fl.item()
        
    #     valid_loss, valid_fl = 0.0, 0.0
    #     md.eval()     # Optional when not using Model Specific layer
    #     for data, labels in valset:
    #         if torch.cuda.is_available():
    #             data, labels = data.cuda(), labels.cuda()

    #         target, logits = md.sampling(data, labels)
    #         kl = beta * torch.mean(md.kl_divergence()) if lr_latent is not None else 0
    #         fl, loss = criterion(target, logits, labels)
    #         loss = loss + kl
    #         valid_fl = fl.item() * data.size(0)
    #         valid_loss = loss.item() * data.size(0)

    #     print(f'Epoch {i+1} \t Focal Loss: {train_fl / len(trainset)} \t Total Loss: {train_loss / len(trainset)}', end='\t -')
    #     print(f'Val Focal Loss: {valid_fl / len(valset)} \t Val Total Loss: {valid_loss / len(valset)}')
    #     if min_valid_loss > valid_fl:
    #         min_valid_loss = valid_fl
    #         # Saving State Dict
    #         torch.save(md.state_dict(), path+temp+"/checkpoint-best.pth")

    # Testing
    md.load_state_dict(torch.load(path+temp+"/checkpoint-best.pth"))

    ## ADNI Testing
    if train == 'ADNI':
        dsc_patient = []
        paths = glob(path_dataset+'ADNI/{}/*'.format('fold'+str(test_idx[0]+1)))
        paths = paths + glob(path_dataset+'ADNI/{}/*'.format('fold'+str(test_idx[1]+1)))
        paths.sort(key=natural_keys)

        img_test = np.concatenate([fold_img[test_idx[0]], fold_img[test_idx[1]]])
        seg_test = np.concatenate([fold_seg[test_idx[0]], fold_seg[test_idx[1]]])

        for idx_p, (img, seg) in enumerate(zip(img_test, seg_test)):
            img = img.transpose(0,3,1,2)
            seg = seg.transpose(0,3,1,2)
            img = torch.Tensor(img); seg = torch.Tensor(seg)

            testset = torch.utils.data.TensorDataset(img,seg)
            testset = torch.utils.data.DataLoader(testset, batch_size=batch_size, 
                                                shuffle=False, num_workers=2)
            # Probabilistic
            if lr_latent is not None:
                y_gen = []
                for x in range(generate_pred):
                    y_pred_patient = []
                    for img_, seg_ in testset:
                        if torch.cuda.is_available():
                            img_, seg_ = img_.cuda(), seg_.cuda()

                        if model_name == 'Prob_MCTrans':
                            y_pred,_ = md.sampling(img_, seg_)
                            y_pred = y_pred.cpu().data.numpy()
                        else:
                            y_pred = md.predict(img_)
                        
                        y_pred_patient.append(y_pred)
                    y_gen.append(np.concatenate(y_pred_patient))
                
                y_gen = np.array(y_gen)
                y_pred = np.average(y_gen, axis=0)
                y_gen = y_gen.transpose(0,1,3,4,2)

            # Deterministic
            else:
                y_pred_patient = []
                for img_, seg_ in testset:
                    if torch.cuda.is_available():
                        img_, seg_ = img_.cuda(), seg_.cuda()

                    if model_name == 'MCTrans':
                        y_pred,_ = md.sampling(img_, seg_)
                        y_pred = y_pred.cpu().data.numpy()
                    else:
                        y_pred = md.predict(img_)
                    
                    y_pred_patient.append(y_pred)
                y_pred = np.concatenate(y_pred_patient)

            # img = img.cpu().data.numpy().transpose(0,2,3,1)
            seg = seg.cpu().data.numpy().transpose(0,2,3,1)
            y_pred = y_pred.transpose(0,2,3,1)
            y_pred[y_pred > threshold] = 1
            y_pred[y_pred <= threshold] = 0
            dsc = dice_coef(seg, y_pred).numpy()
            dsc_patient.append(dsc)
            
            patient_id = paths[idx_p].split('/')[-1]
            fold_id = paths[idx_p].split('/')[-2]
            file_in = '{}/{}_wmh.nii.gz'.format(paths[idx_p], patient_id)
            file_out = result_path.format('ADNI', fold_id, patient_id)
            if len(glob(file_out)) == 0: os.makedirs(file_out)
            file_out_pred = file_out+'/{}_wmh_{}_novl_n_points_6.nii.gz'.format(patient_id, model_name)
            save_wmh(y_pred, file_in, file_out_pred)
            # show_img(img, seg, y_pred, 18)

            # Ambiguity maps
            if lr_latent is not None:
                file_out_1 = file_out+'/{}_am_ss_{}_test.nii.gz'.format(patient_id, model_name)
                file_out_2 = file_out+'/{}_am_sy_{}_test.nii.gz'.format(patient_id, model_name)
                y_gen = torch.sigmoid(torch.Tensor(y_gen))
                y_gen = torch.clamp(y_gen, 1e-6, 1.0 - 1e-6)
                y_gen = y_gen.cpu().data.numpy()
                y_ss = ambiguity_map(y_gen)
                y_sy = ambiguity_map(y_gen, seg)
                save_wmh(y_ss, file_in, file_out_1)
                save_wmh(y_sy, file_in, file_out_2)

    ## Challenge Testing
    else:
        dsc_patient = []
        for iter_,inst in enumerate(['Singapore', 'GE3T', 'Utrecht']):
            dsc_temp = []
            paths = glob(path_dataset+'Challenge/{}/*'.format(inst))
            paths.sort(key=natural_keys)
            paths = paths[10-num:20-num]
            
            img_test = np.array(fold_img[iter_][10-num:20-num], dtype='float32')
            seg_test = np.array(fold_seg[iter_][10-num:20-num], dtype='float32')

            for idx_p, (img, seg) in enumerate(zip(img_test, seg_test)):
                img = img.transpose(0,3,1,2)
                seg = seg.transpose(0,3,1,2)
                img = torch.Tensor(img); seg = torch.Tensor(seg)

                testset = torch.utils.data.TensorDataset(img,seg)
                testset = torch.utils.data.DataLoader(testset, batch_size=batch_size, 
                                                    shuffle=False, num_workers=2)
                
                # Probabilistic
                if lr_latent is not None:
                    y_gen = []
                    for x in range(generate_pred):
                        y_pred_patient = []
                        for img_, seg_ in testset:
                            if torch.cuda.is_available():
                                img_, seg_ = img_.cuda(), seg_.cuda()

                            if model_name == 'Prob_MCTrans':
                                y_pred,_ = md.sampling(img_, seg_)
                                y_pred = y_pred.cpu().data.numpy()
                            else:
                                y_pred = md.predict(img_)
                            
                            y_pred_patient.append(y_pred)
                        y_gen.append(np.concatenate(y_pred_patient))
                    
                    y_gen = np.array(y_gen)
                    y_pred = np.average(y_gen, axis=0)
                    y_gen = y_gen.transpose(0,1,3,4,2)

                # Deterministic
                else:
                    y_pred_patient = []
                    for img_, seg_ in testset:
                        if torch.cuda.is_available():
                            img_, seg_ = img_.cuda(), seg_.cuda()

                        if model_name == 'MCTrans':
                            y_pred,_ = md.sampling(img_, seg_)
                            y_pred = y_pred.cpu().data.numpy()
                        else:
                            y_pred = md.predict(img_)
                        
                        y_pred_patient.append(y_pred)
                    y_pred = np.concatenate(y_pred_patient)

                # img = img.cpu().data.numpy().transpose(0,2,3,1)
                seg = seg.cpu().data.numpy().transpose(0,2,3,1)
                y_pred = y_pred.transpose(0,2,3,1)
                y_pred[y_pred > threshold] = 1
                y_pred[y_pred <= threshold] = 0
                dsc = dice_coef(seg, y_pred).numpy()
                dsc_temp.append(dsc)
                
                patient_id = paths[idx_p].split('/')[-1]
                file_in = '{}/{}_wmh.nii.gz'.format(paths[idx_p], patient_id)
                file_out = result_path.format('Challenge', inst, patient_id)
                if len(glob(file_out)) == 0: os.makedirs(file_out)
                file_out_pred = file_out+'/{}_wmh_{}_novl_n_points_6.nii.gz'.format(patient_id, model_name)
                save_wmh_challenge(y_pred, file_in, file_out_pred, name=inst)
                # show_img(img, seg, y_pred, 28)

                # Ambiguity maps
                if lr_latent is not None:
                    file_out_1 = file_out+'/{}_am_ss_{}.nii.gz'.format(patient_id, model_name)
                    file_out_2 = file_out+'/{}_am_sy_{}.nii.gz'.format(patient_id, model_name)
                    y_gen = torch.sigmoid(torch.Tensor(y_gen))
                    y_gen = torch.clamp(y_gen, 1e-6, 1.0 - 1e-6)
                    y_gen = y_gen.cpu().data.numpy()
                    y_ss = ambiguity_map(y_gen)
                    y_sy = ambiguity_map(y_gen, seg)
                    save_wmh_challenge(y_ss, file_in, file_out_1, name=inst)
                    save_wmh_challenge(y_sy, file_in, file_out_2, name=inst)

            dsc_patient.append(dsc_temp)
    dsc_list.append(dsc_patient)
    print('Average Dice Coefficient on fold-{} : {:.4f}'.format(k+1, np.average(dsc_patient)))
print('Average Dice Coefficient : {:.4f}'.format(np.average(dsc_list)))

Average Dice Coefficient on fold-1 : 0.6153
Average Dice Coefficient on fold-2 : 0.7170
Average Dice Coefficient : 0.6661


# 4.4. Cross Dataset (pytorch model)

## Training

In [ ]:
# ADNI, Challenge
train = 'Challenge'
model_name = 'MCTrans'
# 'ADNI', 'Singapore', 'GE3T', 'Utrecht'
test = ['ADNI']

fold_img, fold_seg = get_dataset(train)
dsc_dict = {i:{j:0 for j in dataset_type} for i in dataset_type}

for type_data in [train]: # train data
    # Training data
    if type_data == 'Challenge':
        img_train = np.concatenate([np.concatenate(i[:-2]) for i in fold_img])
        seg_train = np.concatenate([np.concatenate(i[:-2]) for i in fold_seg])
        img_val = np.concatenate([np.concatenate(i[-2:]) for i in fold_img])
        seg_val = np.concatenate([np.concatenate(i[-2:]) for i in fold_seg])
    else:
        if type_data == 'ADNI':
            img_temp = np.concatenate(fold_img)
            seg_temp = np.concatenate(fold_seg)
        else:
            img_temp = fold_img[0]
            seg_temp = fold_seg[0]

        img_train = np.concatenate(img_temp[:-6])
        seg_train = np.concatenate(seg_temp[:-6])
        img_val = np.concatenate(img_temp[-6:])
        seg_val = np.concatenate(seg_temp[-6:])

    img_train, seg_train = check_data(img_train, seg_train)
    img_val, seg_val = check_data(img_val, seg_val)
    data_augmentation(img_train, seg_train)

    img_train = img_train.transpose(0,3,1,2); seg_train = seg_train.transpose(0,3,1,2) #Full
    img_val = img_val.transpose(0,3,1,2); seg_val = seg_val.transpose(0,3,1,2) #Full

    img_train = torch.Tensor(img_train); seg_train = torch.Tensor(seg_train)
    img_val = torch.Tensor(img_val); seg_val = torch.Tensor(seg_val)

    trainset = torch.utils.data.TensorDataset(img_train,seg_train)
    valset = torch.utils.data.TensorDataset(img_val,seg_val)

    trainset = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                           shuffle=True, num_workers=2, drop_last=True)
    valset = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                         shuffle=False, num_workers=2, drop_last=True)
    
    del img_train, seg_train, img_val, seg_val

    logdir = path+'logdir/{}_{}_novl_n_points_6'.format(model_name, type_data)
    temp = logdir.split('/')[-1]
    # Becareful with these 4 lines!
    ## Will delete existing folders and files
    if len(glob(path+temp)) >= 1: shutil.rmtree(path+temp)
    ## Will create folders and files
    if len(glob(path+temp)) == 0: os.mkdir(path+temp)

    md = pick_model(model_name)
    if torch.cuda.is_available():
        md = md.cuda()
    criterion = TotalLoss(gamma=gamma, alpha=alpha, threshold=threshold)
    optimizer = torch.optim.Adam(md.parameters(), lr=lr)
    min_valid_loss = np.inf

    for i in range(epoch):
        train_loss, train_fl = 0.0, 0.0
        md.train()     # Optional when not using Model Specific layer
        for data, labels in trainset:
            if torch.cuda.is_available():
                data, labels = data.cuda(), labels.cuda()

            optimizer.zero_grad()
            target, logits = md(data, labels)
            
            kl = beta * torch.mean(md.kl_divergence()) if lr_latent is not None else 0
            fl, loss = criterion(target, logits, labels)
            loss = loss + kl
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_fl += fl.item()
        
        valid_loss, valid_fl = 0.0, 0.0
        md.eval()     # Optional when not using Model Specific layer
        for data, labels in valset:
            if torch.cuda.is_available():
                data, labels = data.cuda(), labels.cuda()

            target, logits = md.sampling(data, labels)
            kl = beta * torch.mean(md.kl_divergence()) if lr_latent is not None else 0
            fl, loss = criterion(target, logits, labels)
            loss = loss + kl
            valid_fl = fl.item() * data.size(0)
            valid_loss = loss.item() * data.size(0)

        print(f'Epoch {i+1} \t Focal Loss: {train_fl / len(trainset)} \t Total Loss: {train_loss / len(trainset)}', end='\t -')
        print(f'Val Focal Loss: {valid_fl / len(valset)} \t Val Total Loss: {valid_loss / len(valset)}')
        if min_valid_loss > valid_fl:
            min_valid_loss = valid_fl
            # Saving State Dict
            torch.save(md.state_dict(), path+temp+"/checkpoint-best.pth")

    # # Testing
    # md.load_state_dict(torch.load(path+temp+"/checkpoint-best.pth"))
    # dsc_ = [i for i in dataset_type if i != type_data and i != 'NITRC']

    # for i in test: # test data
    #     score = []
    #     if i == 'ADNI': paths = glob(path_dataset+'ADNI/*/*')
    #     elif i == 'NITRC' : paths = glob(path_dataset+'NITRC/training/*')
    #     else: paths = glob(path_dataset+'Challenge/{}/*'.format(i))
    #     paths.sort(key=natural_keys)
        
    #     img_test, seg_test = get_dataset(i)
    #     img_test = np.concatenate(img_test)
    #     seg_test = np.concatenate(seg_test)

    #     for idx_p, (img, seg) in enumerate(zip(img_test, seg_test)):
    #         img = img.transpose(0,3,1,2)
    #         seg = seg.transpose(0,3,1,2)
    #         img = torch.Tensor(img); seg = torch.Tensor(seg)

    #         testset = torch.utils.data.TensorDataset(img,seg)
    #         testset = torch.utils.data.DataLoader(testset, batch_size=batch_size, 
    #                                               shuffle=False, num_workers=2)
    #         y_pred_list = []
    #         y_ss_list, y_sy_list = [], []
    #         for img_, seg_ in testset:
    #             if torch.cuda.is_available():
    #                 img_, seg_ = img_.cuda(), seg_.cuda()

    #             if lr_latent is not None:
    #                 y_gen = []
    #                 for x in range(generate_pred):
    #                     if model_name == 'Prob_MCTrans':
    #                         y_pred,_ = md.sampling(img_)
    #                         y_pred = y_pred.cpu().data.numpy()
    #                     else:
    #                         y_pred = md.predict(img_)
    #                     y_gen.append(y_pred)

    #                 y_gen = np.array(y_gen)
    #                 y_ss = ambiguity_map(y_gen)
    #                 y_sy = ambiguity_map(y_gen, seg_.cpu().data.numpy())
    #                 y_pred = np.average(y_gen, axis=0)

    #             else:
    #                 if model_name == 'MCTrans':
    #                     y_pred,_ = md(img_, seg_)
    #                     y_pred = y_pred.cpu().data.numpy()
    #                 else:
    #                     y_pred = md.predict(img_)
    #                 y_ss = None; y_sy = None
                    
    #             y_pred[y_pred > threshold] = 1
    #             y_pred[y_pred <= threshold] = 0
    #             y_pred_list.append(y_pred)
    #             if lr_latent is not None:
    #                 y_ss_list.append(y_ss)
    #                 y_sy_list.append(y_sy)
            
    #         y_pred = np.concatenate(y_pred_list)
    #         y_pred = y_pred.transpose(0,2,3,1)
    #         if lr_latent is not None:
    #             y_ss = np.concatenate(y_ss_list)
    #             y_sy = np.concatenate(y_sy_list)
    #             y_ss = y_ss.transpose(0,2,3,1)
    #             y_sy = y_sy.transpose(0,2,3,1)

    #         # img = img.cpu().data.numpy().transpose(0,2,3,1)
    #         seg = seg.cpu().data.numpy().transpose(0,2,3,1)
    #         dsc = dice_coef(seg, y_pred).numpy()
    #         score.append(dsc)

    #         patient_id = paths[idx_p].split('/')[-1]
    #         if i == 'ADNI':
    #             fold_id = paths[idx_p].split('/')[-2]
    #             file_out = result_path.format(i, fold_id, patient_id)
    #         elif i == 'NITRC':
    #             fold_id = 'training'
    #             file_out = result_path.format(i, fold_id, patient_id)
    #         else:
    #             fold_id = 'Challenge'
    #             file_out = result_path.format(fold_id, i, patient_id)

    #         temp = 'lesion' if i == 'NITRC' else 'wmh'
    #         file_in = '{}/{}_{}.nii.gz'.format(paths[idx_p], patient_id, temp)
    #         if len(glob(file_out)) == 0: os.makedirs(file_out)
    #         file_out_pred = file_out+'/{}_wmh_{}_{}.nii.gz'.format(patient_id, model_name,type_data)
    #         save_wmh(y_pred, file_in, file_out_pred, name=i)
    #         # show_img(img, seg, y_pred, 18)

    #         # Ambiguity maps
    #         if y_ss is not None and y_sy is not None:
    #             file_out_1 = file_out+'/{}_am_ss_{}_{}.nii.gz'.format(patient_id, model_name, type_data)
    #             file_out_2 = file_out+'/{}_am_sy_{}_{}.nii.gz'.format(patient_id, model_name, type_data)
    #             save_wmh(y_ss, file_in, file_out_1, name=i)
    #             save_wmh(y_sy, file_in, file_out_2, name=i)

    #     dsc_dict[type_data][i] = np.average(score)
    #     print('Dice Coefficient with dataset {} trained by {}: {:.4f}'.format(i, type_data, dsc_dict[type_data][i]))
    # avg_ = np.average([dsc_dict[train][i] for i in test])
    # print('Average: {:.4f}'.format(avg_))

Epoch 1 	 Focal Loss: 0.005928911717554755 	 Total Loss: 0.0766727959626559	 -Val Focal Loss: 2.744446557237586e-05 	 Val Total Loss: 0.004759978002576686
Epoch 2 	 Focal Loss: 0.0020821843313854516 	 Total Loss: 0.07199485993452287	 -Val Focal Loss: 1.2353947038637168e-05 	 Val Total Loss: 0.00412605769598662
Epoch 3 	 Focal Loss: 0.0015248975813676405 	 Total Loss: 0.071043419967111	 -Val Focal Loss: 1.2691158787416878e-05 	 Val Total Loss: 0.004429088154835488
Epoch 4 	 Focal Loss: 0.0014005334963040545 	 Total Loss: 0.07103456427039725	 -Val Focal Loss: 5.855972169717746e-06 	 Val Total Loss: 0.004113591429012925
Epoch 5 	 Focal Loss: 0.0012911004927545172 	 Total Loss: 0.07080224430843303	 -Val Focal Loss: 4.5459609784519495e-06 	 Val Total Loss: 0.004157048107972785
Epoch 6 	 Focal Loss: 0.0012288394120615882 	 Total Loss: 0.0707924859912208	 -Val Focal Loss: 5.4278787149032995e-06 	 Val Total Loss: 0.004361250062487019
Epoch 7 	 Focal Loss: 0.0011473382629281896 	 Total Loss: 0.

## Testing

In [ ]:
# ADNI, Challenge
train = 'Challenge'
model_name = 'MCTrans'
# 'ADNI', 'Singapore', 'GE3T', 'Utrecht'
test = ['ADNI']

# fold_img, fold_seg = get_dataset(train)
dsc_dict = {i:{j:0 for j in dataset_type} for i in dataset_type}

for type_data in [train]: # train data
    # # Training data
    # if type_data == 'Challenge':
    #     img_train = np.concatenate([np.concatenate(i[:-2]) for i in fold_img])
    #     seg_train = np.concatenate([np.concatenate(i[:-2]) for i in fold_seg])
    #     img_val = np.concatenate([np.concatenate(i[-2:]) for i in fold_img])
    #     seg_val = np.concatenate([np.concatenate(i[-2:]) for i in fold_seg])
    # else:
    #     if type_data == 'ADNI':
    #         img_temp = np.concatenate(fold_img)
    #         seg_temp = np.concatenate(fold_seg)
    #     else:
    #         img_temp = fold_img[0]
    #         seg_temp = fold_seg[0]

    #     img_train = np.concatenate(img_temp[:-6])
    #     seg_train = np.concatenate(seg_temp[:-6])
    #     img_val = np.concatenate(img_temp[-6:])
    #     seg_val = np.concatenate(seg_temp[-6:])

    # img_train, seg_train = check_data(img_train, seg_train)
    # img_val, seg_val = check_data(img_val, seg_val)
    # data_augmentation(img_train, seg_train)

    # img_train = img_train.transpose(0,3,1,2); seg_train = seg_train.transpose(0,3,1,2) #Full
    # img_val = img_val.transpose(0,3,1,2); seg_val = seg_val.transpose(0,3,1,2) #Full

    # img_train = torch.Tensor(img_train); seg_train = torch.Tensor(seg_train)
    # img_val = torch.Tensor(img_val); seg_val = torch.Tensor(seg_val)

    # trainset = torch.utils.data.TensorDataset(img_train,seg_train)
    # valset = torch.utils.data.TensorDataset(img_val,seg_val)

    # trainset = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
    #                                        shuffle=True, num_workers=2, drop_last=True)
    # valset = torch.utils.data.DataLoader(valset, batch_size=batch_size,
    #                                      shuffle=False, num_workers=2, drop_last=True)
    
    # del img_train, seg_train, img_val, seg_val

    logdir = path+'logdir/{}_{}_novl_n_points_6'.format(model_name, type_data)
    temp = logdir.split('/')[-1]
    # # Becareful with these 4 lines!
    # ## Will delete existing folders and files
    # if len(glob(path+temp)) >= 1: shutil.rmtree(path+temp)
    # ## Will create folders and files
    # if len(glob(path+temp)) == 0: os.mkdir(path+temp)

    md = pick_model(model_name)
    if torch.cuda.is_available():
        md = md.cuda()
    # criterion = TotalLoss(gamma=gamma, alpha=alpha, threshold=threshold)
    # optimizer = torch.optim.Adam(md.parameters(), lr=lr)
    # min_valid_loss = np.inf

    # for i in range(epoch):
    #     train_loss = 0.0
    #     md.train()     # Optional when not using Model Specific layer
    #     for data, labels in trainset:
    #         if torch.cuda.is_available():
    #             data, labels = data.cuda(), labels.cuda()

    #         optimizer.zero_grad()
    #         target, logits = md(data, labels)
            
    #         kl = beta * torch.mean(md.kl_divergence()) if lr_latent is not None else 0
    #         loss = criterion(target, logits, labels) + kl
    #         loss.backward()
    #         optimizer.step()
    #         train_loss += loss.item()
        
    #     valid_loss = 0.0
    #     del data, labels
    #     md.eval()     # Optional when not using Model Specific layer
    #     for data, labels in valset:
    #         if torch.cuda.is_available():
    #             data, labels = data.cuda(), labels.cuda()

    #         target, logits = md.sampling(data)
    #         kl = beta * torch.mean(md.kl_divergence()) if lr_latent is not None else 0
    #         loss = criterion(target, logits, labels) + kl
    #         valid_loss = loss.item() * data.size(0)

    #     print(f'Epoch {i+1} \t\t Training Loss: {train_loss / len(trainset)} \t\t Validation Loss: {valid_loss / len(valset)}')
    #     if min_valid_loss > valid_loss:
    #         min_valid_loss = valid_loss
    #         # Saving State Dict
    #         torch.save(md.state_dict(), path+temp+"/checkpoint-best.pth")

    # Testing
    md.load_state_dict(torch.load(path+temp+"/checkpoint-best.pth"))
    dsc_ = [i for i in dataset_type if i != type_data and i != 'NITRC']

    for i in test: # test data
        score = []
        if i == 'ADNI': paths = glob(path_dataset+'ADNI/*/*')
        elif i == 'NITRC' : paths = glob(path_dataset+'NITRC/training/*')
        else: paths = glob(path_dataset+'Challenge/{}/*'.format(i))
        paths.sort(key=natural_keys)
        
        img_test, seg_test = get_dataset(i)
        img_test = np.concatenate(img_test)
        seg_test = np.concatenate(seg_test)

        for idx_p, (img, seg) in enumerate(zip(img_test, seg_test)):
            img = img.transpose(0,3,1,2)
            seg = seg.transpose(0,3,1,2)
            img = torch.Tensor(img); seg = torch.Tensor(seg)

            testset = torch.utils.data.TensorDataset(img,seg)
            testset = torch.utils.data.DataLoader(testset, batch_size=batch_size, 
                                                  shuffle=False, num_workers=2)
            # Probabilistic
            if lr_latent is not None:
                y_gen = []
                for x in range(generate_pred):
                    y_pred_patient = []
                    for img_, seg_ in testset:
                        if torch.cuda.is_available():
                            img_, seg_ = img_.cuda(), seg_.cuda()

                        if model_name == 'Prob_MCTrans':
                            y_pred,_ = md.sampling(img_, seg_)
                            y_pred = y_pred.cpu().data.numpy()
                        else:
                            y_pred = md.predict(img_)
                        
                        y_pred_patient.append(y_pred)
                    y_gen.append(np.concatenate(y_pred_patient))
                
                y_gen = np.array(y_gen)
                y_pred = np.average(y_gen, axis=0)
                y_gen = y_gen.transpose(0,1,3,4,2)

            # Deterministic
            else:
                y_pred_patient = []
                for img_, seg_ in testset:
                    if torch.cuda.is_available():
                        img_, seg_ = img_.cuda(), seg_.cuda()

                    if model_name == 'MCTrans':
                        y_pred,_ = md.sampling(img_, seg_)
                        y_pred = y_pred.cpu().data.numpy()
                    else:
                        y_pred = md.predict(img_)
                    
                    y_pred_patient.append(y_pred)
                y_pred = np.concatenate(y_pred_patient)

            # img = img.cpu().data.numpy().transpose(0,2,3,1)
            seg = seg.cpu().data.numpy().transpose(0,2,3,1)
            y_pred = y_pred.transpose(0,2,3,1)
            y_pred[y_pred > threshold] = 1
            y_pred[y_pred <= threshold] = 0
            dsc = dice_coef(seg, y_pred).numpy()
            score.append(dsc)

            patient_id = paths[idx_p].split('/')[-1]
            if i == 'ADNI':
                fold_id = paths[idx_p].split('/')[-2]
                file_out = result_path.format(i, fold_id, patient_id)
            elif i == 'NITRC':
                fold_id = 'training'
                file_out = result_path.format(i, fold_id, patient_id)
            else:
                fold_id = 'Challenge'
                file_out = result_path.format(fold_id, i, patient_id)

            temp = 'lesion' if i == 'NITRC' else 'wmh'
            file_in = '{}/{}_{}.nii.gz'.format(paths[idx_p], patient_id, temp)
            if len(glob(file_out)) == 0: os.makedirs(file_out)
            file_out_pred = file_out+'/{}_wmh_{}_{}_novl_n_points_6.nii.gz'.format(patient_id, model_name,type_data)
            save_wmh(y_pred, file_in, file_out_pred, name=i)
            # show_img(img, seg, y_pred, 18)

            # Ambiguity maps
            if lr_latent is not None:
                file_out_1 = file_out+'/{}_am_ss_{}_{}.nii.gz'.format(patient_id, model_name, type_data)
                file_out_2 = file_out+'/{}_am_sy_{}_{}.nii.gz'.format(patient_id, model_name, type_data)
                y_gen = torch.sigmoid(torch.Tensor(y_gen))
                y_gen = torch.clamp(y_gen, 1e-6, 1.0 - 1e-6)
                y_gen = y_gen.cpu().data.numpy()
                y_ss = ambiguity_map(y_gen)
                y_sy = ambiguity_map(y_gen, seg)
                save_wmh(y_ss, file_in, file_out_1, name=i)
                save_wmh(y_sy, file_in, file_out_2, name=i)

        dsc_dict[type_data][i] = np.average(score)
        print('Dice Coefficient with dataset {} trained by {}: {:.4f}'.format(i, type_data, dsc_dict[type_data][i]))
    avg_ = np.average([dsc_dict[train][i] for i in test])
    print('Average: {:.4f}'.format(avg_))

Dice Coefficient with dataset ADNI trained by Challenge: 0.4071
Average: 0.4071


# 5. Evaluation

In [ ]:
# 'ADNI', 'Challenge'
train = 'Challenge'
# 'ADNI', 'Singapore', 'GE3T', 'Utrecht', 'Challenge'
test = ['ADNI']
# U_Net, AttU_Net, XNet, AttXNet, KiU_Net, Prob_U_Net, H_Prob_U_Net, PHiSeg
model_name = 'MCTrans'
df_by_model, dsc, patient = [], [], []

for i in test: # testing 
    if i == 'ADNI': 
        paths = glob(path_dataset+'ADNI/*/*/*wmh.nii.gz')
    elif i == 'NITRC' : 
        paths = glob(path_dataset+'NITRC/training/*/*lesion.nii.gz')
    elif i == 'Challenge':
        paths = []
        for j in ['Singapore', 'GE3T', 'Utrecht']:
            path = glob(path_dataset+'Challenge/{}/*/*wmh.nii.gz'.format(j))
            path.sort(key=natural_keys)
            paths += path
    else: 
        paths = glob(path_dataset+'Challenge/{}/*/*wmh.nii.gz'.format(i))

    if i != 'Challenge': paths.sort(key=natural_keys)
    temp, temp_dsc, temp_patient = [], [], []

    for lp in paths:
        patient_id = lp.split('/')[-2]
        if i == 'ADNI':
            fold_id = lp.split('/')[-3]
            pp = result_path.format(i, fold_id, patient_id)
        elif i == 'NITRC':
            pp = result_path.format(i, 'training', patient_id)
        elif i == 'Challenge':
            fold_id = lp.split('/')[-3]
            pp = result_path.format('Challenge', fold_id, patient_id)
        else:
            pp = result_path.format('Challenge', i, patient_id)
        
        if i == train:
            pp = pp+'/{}_wmh_{}_novl_n_points_6.nii.gz'.format(patient_id, model_name) # training
        else:
            pp = pp+'/{}_wmh_{}_{}_novl_n_points_6.nii.gz'.format(patient_id, model_name, train) # training

        voxel_true = nib.load(lp); wmh_true = read_data(lp)
        voxel_pred = nib.load(pp); wmh_pred = read_data(pp)
        volume_true = np.prod([abs(voxel_true.affine[i][i]) for i in range(3)])/1000
        volume_true = np.count_nonzero(wmh_true)*volume_true
        volume_pred = np.prod([abs(voxel_pred.affine[i][i]) for i in range(3)])/1000
        volume_pred = np.count_nonzero(wmh_pred)*volume_pred
        score = dice_coef(wmh_true, wmh_pred).numpy()
        temp_dsc.append(score)
        temp_patient.append(patient_id)
        temp.append([volume_true, volume_pred])
    df_by_model.append(temp)
    dsc.append(temp_dsc)
    patient.append(temp_patient)

df_temp = [pd.DataFrame(i, index=j).transpose() for i,j in zip(df_by_model, patient)]
temp_str = '_Cross' if train not in test else ''
with pd.ExcelWriter(eval_path+'output{}_{}_{}_novl_n_points_6.xlsx'.format(temp_str, train, model_name)) as writter:
    num = 0
    for k,i in enumerate(test):
        df_temp[k].to_excel(writter, startrow=num, startcol=0, sheet_name='vol')
        pd.DataFrame([dsc[k]], columns=df_temp[k].columns).to_excel(
            writter, startrow=num, startcol=0, sheet_name='dsc')
        num = num + df_temp[k].shape[0] + 3